Replicating the paper using CNN regression model

In [ ]:
""" 
This cell is the main part of our work where we have replicated the results that are obtained by
'Deep learning applied to CO2 power plant emissions quantification using simulated satellite images' (Joffrey Dumont Le Brazidec et al.). Our main
aim is to ensure that we have the understanding of the model (which they have mentioned in their paper) and replicate the results. We understand that absolute
replication is very difficult as mentioned by author of the paper due to the random initialization of weights, parameters and hyperparameters by optimization
algorithms and hence have followed the similar model ensembling approach as mentioned. Note for the reader : In the paper, authors mentioned running the
model at a particular location for a particular additional input (None, Segmentation, NO2) for 3 times. Due to computation related issues, we
ran it for 2 times and have got reasonable results to believe that our work is quite satisfactorily close to the results mentioned in Table 2 of the paper.

"""

"""
Importing different packages that we will use.

"""
import h5py
import os
import numpy as np
from netCDF4 import Dataset as NetCDFDataset
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, LeakyReLU, Dropout, BatchNormalization, MaxPool2D, Flatten, Dense, GaussianNoise
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tqdm import tqdm
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
import pandas as pd
import tensorflow as tf

"""
We define our Custom Dataset which will read the .nc files (train, valid, test) and extract the XCO2, u, v, NO2 data from the .nc files.
Note to the reader : For our comparision, we are going to consider Lippendorf location and as additional input we will consider NO2 images.

"""
class CustomDataset:
    
    def __init__(self, file_path, variable = None):
        self.ncfile = NetCDFDataset(file_path, 'r') # Reading the .nc file
        self.variable = variable # Variable in our comparison is no2_noisy
    
    def __len__(self):
        return self.ncfile.variables['xco2_noisy'].shape[0] # The total number of images or data we will have which is different for train (25152), valid (4608), test (6289)
    
    def __getitem__(self, idx):

        # Reading the necessary variables (xco2_noisy, u, v)
        xco2_read = self.ncfile.variables['xco2_noisy'][idx] # Unit : ppmv (parts per million by volume), Shape : 64, 64
        u_read = self.ncfile.variables['u'][idx] # Unit : m/s, Shape : 64, 64
        v_read = self.ncfile.variables['v'][idx] # Unit : m/s, Shape : 64, 64
        emiss_read = self.ncfile.variables['emiss'][idx] # emiss is the true output data that we need to train our model. Unit : Mt/yr (Million tonnes/ year)
                                                         # Shape : 3,
        # Converting into numpy arrays with data type float32
        xco2_arr = np.array(xco2_read.data).astype('float32') 
        u_arr = np.array(u_read.data).astype('float32')
        v_arr = np.array(v_read.data).astype('float32')
        emiss_arr = np.array(emiss_read.data).astype('float32')

        #noise = GaussianNoise(stddev = 0.7) # Addition of Gaussian Noise with std of 0.7
        #xco2 = noise(xco2_arr).numpy() 
        if self.variable:
            var_read = self.ncfile.variables[self.variable][idx] # Unit : molec/cm^2, Shape : 64, 64
            var_arr = np.array(var_read.data).astype('float32') # Converting into numpy array after reading the variable
            #var_noise = noise(var_tensor).numpy()
            inputs = np.stack([xco2_arr, u_arr, v_arr, var_arr], axis = -1) # Stacking the inputs to get the desired input shape = (64, 64, 4) where 64, 64
                                                                            # represents height, width and 4 represents the total number of features. Each pixel
                                                                            # has a spatial resolution of 2km, i.e. 1*1 in pixel refers to an area of 2km*2km.
        else: 
            inputs = np.stack([xco2_arr, u_arr, v_arr], axis = -1) # Stacking the inputs to get the shape = (64, 64, 3) in case when variable is None.
        
        mean = inputs.mean(axis = (0, 1), keepdims = True) # Calculating the mean of the 4 features separately and keeping the same shape as that of inputs.
        
        std = inputs.std(axis = (0, 1), keepdims = True) # Calculating the std of the 4 features separately and keeping the same shape as that of inputs.
        
        std[std==0] = 1 # If std becomes 0, we change it 1 to avoid division by 0. Multiple other methods can also be employed such as choosing an 
                        # appropriate epsilon such as 1e-5, 1e-6 or 1e-7
        
        inputs = (inputs - mean)/std # Normalization of the inputs separately for each feature.
        
        # emiss data is of shape (3,), from which we will only consider the middle value for our comparison. Hence, we multiply weights with emiss to
        # get the middle value and round it off to 3 decimal places.
        weights = np.array([0.0, 1.0, 0.0]) 
        weighted = np.round(np.sum(weights * emiss_arr), 3)
        outputs = np.array([weighted], dtype = np.float32)
        
        return inputs, outputs # returns our inputs (64, 64, 4), outputs (1,)

    def __del__(self):
        self.ncfile.close() # Closing the ncfile to avoid corruption of the file.

"""
The model architecture is taking from the paper which consists of 2D convolution, Maxpooling, Dropout, Batch Normalization, Flatten and finally Dense.

"""
def model_arch(input_shape): # input_shape = 64, 64, 4
    model = Sequential()
    
    # The first convolution with kernel size of (3,3), total number of filters = 32, activation function = elu (Exponential Linear Unit), strides = 1, 
    # padding = valid (no padding).
    model.add(Conv2D(32, (3, 3), activation = "elu", strides = 1, input_shape = input_shape)) # Shape after conv : 62, 62, 32
    
    # Adding a Dropout of 0.1
    model.add(Dropout(0.1)) # No change in shape
    
    # Second convolution with kernel size of (3,3), total number of filters = 32, activation function = elu (Exponential Linear Unit), strides = 1,
    # padding = valid (no padding)
    model.add(Conv2D(32, (3, 3), activation = "elu", strides = 1)) # Shape after conv : 60, 60, 32
    
    # Maxpool with pool size of 2, 2 and strides = 2.
    model.add(MaxPool2D(pool_size = (2, 2), padding = "valid", strides = 2)) # Shape after maxpool : 30, 30, 32
    
    model.add(BatchNormalization()) # Batch Normalization which does not affect the shape
    
    # Third convolution with kernel size of (3,3), total number of filters = 32, activation function = elu (Exponential Linear Unit), strides = 1,
    # padding = valid (no padding)
    model.add(Conv2D(32, (3, 3), activation = "elu", strides = 1)) # Shape after conv : 28, 28, 32
    
    # Adding a Dropout of 0.2
    model.add(Dropout(0.2))
    
    # Fourth convolution with kernel size of (3,3), total number of filters = 64, activation function = elu (Exponential Linear Unit), strides = 1,
    # padding = valid (no padding)
    model.add(Conv2D(64, (3, 3), activation = "elu", strides = 1)) # Shape after conv : 26, 26, 64
    
    model.add(BatchNormalization()) # Batch Normalization which does not affect the shape
    
    # Fifth convolution with kernel size of (3,3), total number of filters = 64, activation function = elu (Exponential Linear Unit), strides = 1,
    # padding = valid (no padding)
    model.add(Conv2D(64, (3, 3), activation = "elu", strides = 1)) # Shape after conv : 24, 24, 64
    
    # Adding a Dropout of 0.2
    model.add(Dropout(0.2)) # No change in shape
    
    # Sixth convolution with kernel size of (3,3), total number of filters = 64, activation function = elu (Exponential Linear Unit), strides = 1,
    # padding = valid (no padding)
    model.add(Conv2D(64, (3, 3), activation = "elu", strides = 1)) # Shape after conv : 22, 22, 64
    
    # Maxpool with pool size of 2, 2 and strides = 2.
    model.add(MaxPool2D(pool_size = (2, 2), padding = "valid", strides = 2)) # Shape after maxpool : 11, 11, 64
    
    # Seventh convolution with kernel size of (3,3), total number of filters = 64, activation function = elu (Exponential Linear Unit), strides = 1,
    # padding = valid (no padding)
    model.add(Conv2D(64, (3, 3), activation = "elu", strides = 1)) # Shape after conv : 9, 9, 64
    
    # Adding a Dropout of 0.2
    model.add(Dropout(0.2))
    
    # Eighth convolution with kernel size of (3,3), total number of filters = 64, activation function = elu (Exponential Linear Unit), strides = 1,
    # padding = valid (no padding)
    model.add(Conv2D(64, (3, 3), activation = "elu", strides = 1)) # Shape after conv : 7, 7, 64
    
    # Maxpool with pool size of 2, 2 and strides = 2.
    model.add(MaxPool2D(pool_size = (2, 2), padding = "valid", strides = 2)) # Shape after maxpool : 3, 3, 64
    
    model.add(Flatten()) # Total = 3*3*64 = 576
    
    model.add(Dense(1)) # Fully Connected Layer to get a single output.
    
    model.add(LeakyReLU(alpha = 0.3)) # The output goes through activation function = Leaky Rectified Linear Unit with negative slope = 0.3.
    
    return model

# Creating and checking if saved_models folder exists or not. If not, then create a folder where we will store our weights.h5 file.
checkpoint_dir = './saved_models/' 
os.makedirs(checkpoint_dir, exist_ok = True)

best_model_filepath = os.path.join(checkpoint_dir, 'best_model_weights.h5')
# Callbacks can be of best weights or last weights. We can do either one. We went with saving the weights after every epoch. 
best_model_checkpoint_callback = ModelCheckpoint(filepath = best_model_filepath, save_weights_only = True, verbose = 1)

# Applying data augmentation to Training data as specified by the paper.
data_augmentation = ImageDataGenerator(rotation_range = 180, width_shift_range = 0.0, height_shift_range = 0.0, shear_range = 90, zoom_range = 0.2, horizontal_flip = True, vertical_flip = True, fill_mode = 'nearest')

# Filepaths for Train, Valid, Test. From the paper we know that for Train, Valid we will choose data points from everywhere except the location in which
# we will test our model's performance. Location = Lippendorf
test_set = 'curated/lip_test_dataset.nc'
valid_set = 'curated/lip_valid_dataset.nc'
train_set = 'curated/lip_train_dataset.nc'

# Here, we have generated our custom dataset.
train_dataset = CustomDataset(train_set, variable = 'no2_noisy')
valid_dataset = CustomDataset(valid_set, variable = 'no2_noisy')
test_dataset = CustomDataset(test_set, variable = 'no2_noisy')

# To store the inputs and outputs from our dataset to feed it to the model during training, validation and inference.
train_inputs = []
train_outputs = []
val_inputs = []
val_outputs = []
test_inputs = []
test_outputs = []

for inputs, outputs in train_dataset:
    train_inputs.append(inputs)
    train_outputs.append(outputs)
for inputs, outputs in valid_dataset:
    val_inputs.append(inputs)
    val_outputs.append(outputs)
for inputs, outputs in test_dataset:
    test_inputs.append(inputs)
    test_outputs.append(outputs)

# Converting into numpy arrays
train_inputs = np.array(train_inputs) # train_inputs shape : (25152, 64, 64, 4)
train_outputs = np.array(train_outputs) # train_outputs shape : (25152, 1)
val_inputs = np.array(val_inputs) # val_inputs shape : (4608, 64, 64, 4)
val_outputs = np.array(val_outputs) # val_outputs shape : (4608, 1)
test_inputs = np.array(test_inputs) # test_inputs shape : (6289, 64, 64, 4)
test_outputs = np.array(test_outputs) # test_outputs shape : (6289, 1)

model = model_arch(input_shape = (64, 64, 4)) # Declaring the model

#model.load_weights('saved_models/best_model_weights.h5') # If we have already stored the weights, then uncomment this line to load.

optimizer = Adam(learning_rate = 1e-3) # Taking Adam optimizer with lr = 1e-3

# We will update our lr if we reach a plateau. For this we use ReduceLROnPlateau which will monitor val_loss, wait for 20 epochs before changing lr and measures 
# min_delta threshold, reduces lr by 0.5 and lower bound on lr is 5e-5.
learning_rate_monitor_callback = ReduceLROnPlateau(monitor = 'val_loss', factor = 0.5, patience = 20, verbose = 1, min_delta = 5e-3, cooldown = 0, min_lr = 5e-5)

# Loss function for this task is MeanAbsoluteError and metrics is MeanAbsolutePercentageError.
model.compile(optimizer = optimizer, loss = 'mae', metrics = ['mape'])

# Generate the summary of the model. Around 186000 trainable paramaters are there.
model.summary()

# Total epochs = 500, batch size for training set = 32, steps will be 25152/32 = 786, validation will not have any augmentation with batch size of 32 and validating
# on the entire validation set.
history = model.fit(data_augmentation.flow(train_inputs, train_outputs, batch_size = 32, shuffle = True), epochs = 500, steps_per_epoch = len(train_dataset)//32,
validation_data = (val_inputs, val_outputs), validation_batch_size = 32, validation_steps = None, callbacks = [learning_rate_monitor_callback, best_model_checkpoint_callback])

# To test the model's performance on test set.
#test_loss, test_mae = model.evaluate(test_inputs, test_outputs, batch_size = None) # Uncomment these lines to infer model's performance.
#print(f'Test Loss: {test_loss}, Test MAE: {test_mae}')

# To store the true_emission and predicted emissions for the .csv file.
true_emissions = []
predicted_emissions = []
for inputs, targets in test_dataset:
    inputs = np.expand_dims(inputs, axis = 0)
    outputs = model.predict(inputs) # Predicting the output based on the model's learning.
    print("True value : ", targets)
    print("Predicted value : ", outputs)
    true_emissions.append(targets) # Append the true emissions into true_emissions list
    predicted_emissions.append(outputs) # Append the predicted emissions into predicted_emissions list

# Converting into numpy arrays
true_emissions = np.array(true_emissions) # Shape : 6289, 3
predicted_emissions = np.array(predicted_emissions) # Shape : 6289, 1, 3
true_emissions = true_emissions.reshape(predicted_emissions.shape) # After reshaping : 6289, 1, 3

# Storing the true emissions and predicted emissions in a csv file. 
df = pd.DataFrame({'True Emissions': true_emissions.flatten(),'Predicted Emissions': predicted_emissions.flatten()}, index = np.arange(1, len(true_emissions) + 1))
df.to_csv('emissions_data_lip.csv')
print(df)


In [ ]:
# importing necessary package
import pandas as pd

data_df = pd.read_csv('emissions_data_lip.csv') # Reading emissions_data

# Creating new column error which is true-pred.
data_df['error'] = df['True Emissions'] - df['Predicted Emissions'] 

# Creating new column absolute_error which is the absolute value of error rounded off to 3 decimal places.
data_df['absolute_error'] = (abs(df['error'])).round(3)

# Creating new column relative_error which is the relative value of error rounded off to 3 decimal places with epsilon = 1e-15.
data_df['relative_error'] = ((df['error'] / (df['True Emissions'] + 1e-15)) * 100).round(3)

# Updates the csv file
data_df.to_csv('emissions_data_lip.csv', index = False)

In [ ]:
# importing necessary package
import pandas as pd

df = pd.read_csv('emissions_data_lip.csv') # Reading emissions_data

# We want to know the Q1, Q2, Q3 values or (25%, 50%, 75%) values from our absolute_error distribution.
percentiles = df['absolute_error'].describe(percentiles = [0.25, 0.5, 0.75])
percentiles = percentiles.loc[['25%', '50%', '75%']].apply(abs) # 25% : 2.123, 50% : 4.058, 75% : 6.426 (All are in Mt/yr)

# Stated in the paper, the median absolute error is 3 Mt/yr. We got 4 Mt/ yr which means we are at a 1 Mt/yr deviation which is quite reasonable.

# The true percentile values as mentioned in the paper.
true_percentiles = [1.3, 2.7, 4.5] # (All are in Mt/yr)

# Absolute error between our percentile values and the true percentile values rounded off to 3 decimal places.
error = [round(abs(true - pred), 3) for true, pred in zip(true_percentiles, percentiles)] # 0.823, 1.358, 1.926

# This means that our percentile values are within +2 Mt/yr range of the true percentile values. This is pretty reasonable considering the random nature of
# initialization of weights, parameters and hyperparameters by the optimization algorithms. 

print(percentiles)
print("Error:", error)

In [ ]:
# importing necessary package
import pandas as pd

df = pd.read_csv('emissions_data_lip.csv') # Reading emissions_data
df['abs_rel_error'] = abs(df['relative_error'])
# We want to know the Q1, Q2, Q3 values or (25%, 50%, 75%) values from our relative_error distribution.
percentiles = df['abs_rel_error'].describe(percentiles = [0.25, 0.5, 0.75])

# Only the percentiles are of impportant to us.
percentiles = percentiles.loc[['25%', '50%', '75%']].apply(abs) # 25% : 8.151, 50% : 19.183, 75% : 37.049 (Unitless)

# Stated in the paper, the median absolute relative paper is around 20%. We have got 19.183% which can be approximated to 20%.

# The true percentile values as mentioned in the paper.
true_percentiles = [8.6, 18.1, 30.3] # Note : These values are percentage values as the relative errors are calculated in terms of percentage.

# Absolute error between our percentile values and the true percentile values rounded off to 3 decimal places.
error = [round(abs(true - pred), 3) for true, pred in zip(true_percentiles, percentiles)] # 0.449, 1.083, 6.749

# Again we notice a +2% range deviation (barring only one value) which can be associated to the fact that the model ran for only 2 times compared to 3 times
# mentioned in the paper. Also, the random initialization of weights, parameters and hyperparameters by the optimization algorithms may have played a role here.
print(percentiles)
print("Error:", error)

In [ ]:
""" This cell is for plotting the Kernel Density Estimation (KDE) plots using seaborn of absolute_error and relative_error. """

# importing necessary packages
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

df = pd.read_csv('emissions_data_lip.csv') # Reading emissions_data

plt.figure(figsize = (10, 6))
sns.kdeplot(df['absolute_error'], shade = True, color = 'b') # Using seaborn's kdeplot. 
plt.title('Kernel Density Estimate of Absolute Error')
plt.xlabel('Absolute Error')
plt.ylabel('Density')
plt.savefig('absolute_error_plot.png', dpi = 300, bbox_inches = 'tight') # Saving the file
plt.show()

plt.figure(figsize = (10, 6))
sns.kdeplot(df['relative_error'], shade = True, color = 'r') # Using seaborn's kdeplot.
plt.title('Kernel Density Estimate of Relative Error (%)')
plt.xlabel('Relative Error (%)')
plt.ylabel('Density')
plt.savefig('relative_error_plot.png', dpi = 300, bbox_inches = 'tight') # Saving the file
plt.show()


In [ ]:
""" This code is to know how many of our absolute errors lie in (0-2) Mt/yr range, in (2-5) Mt/yr range, in (5-10) Mt/yr range
and in 10 Mt/yr above. Along with that, let us also get some statistics. """

# importing necessary package
import pandas as pd

# Read csv file
df = pd.read_csv('emissions_data.csv')

# useful column
abs = df['absolute_error']

# get the required info
in0_2 = (abs >= 0) & (abs <= 2)
in2_5 = (abs > 2) & (abs <= 5)
in5_10 = (abs > 5) & (abs <= 10)
ab10 = abs > 10
mean = round(abs.mean(), 3)
median = round(abs.median(), 3)
std = round(abs.std(), 3)

print("in between 0 and 2 : ", in0_2.sum()) # 1477
print("in between 2 and 5 : ", in2_5.sum()) # 2358
print("in between 5 and 10 : ", in5_10.sum()) # 2172
print("above 10 : ", ab10.sum()) # 282
print("mean : ", mean) # 4.474
print("median : ", median) # 4.058
print("std : ", std) # 2.967

# According to the paper, " Majority of errors are concentrated below 10 Mt/yr." which we can see is true.

In [ ]:
""" This code is to know how many of our relative errors lie less than -150%, in (-150 to -100)% range, in (-100 to -50)% range,
in (-50 to 0) %, in (0 to 50) % range, in (50 to 100) % and above 100%. Along with that, let us also get some statistics. """

# importing necessary package
import pandas as pd

# Read csv file
df = pd.read_csv('emissions_data.csv')

# useful column
rel = df['relative_error']

# get the required info
lessneg150 = (rel <= -150)
neg150_100 = (rel > -150) & (rel <= -100)
neg100_50 = (rel > -100) & (rel <= -50)
neg50_0 = (rel > -50) & (rel <= 0)
in0_50 = (rel > 0) & (rel <= 50)
in50_100 = (rel > 50) & (rel <= 100)
ab100 = rel > 100
mean = round(rel.mean(), 3)
median = round(rel.median(), 3)
std = round(rel.std(), 3)

print("less than - 150% : ", lessneg150.sum()) # 1
print("in between -150% and -100% : ", neg150_100.sum()) # 38
print("in between -100% and -50% : ", neg100_50.sum()) # 257
print("in between -50% and 0% : ", neg50_0.sum()) # 1662
print("in between 0% and 50% : ", in0_50.sum()) # 3892
print("in between 50% and 100% : ", in50_100.sum()) # 439
print("above 100% : ", ab100.sum()) # 0
print("mean : ", mean) # 11.938
print("median : ", median) # 19.183
print("std : ", std) # 32.561

In [ ]:
""" This code is to know how many of our absolute relative errors lie in (0 to 20) % range, in (20 to 50)% range, in (50 to 100) %, in (100 to 150)% 
and above 150%. Along with that, let us also get some statistics. """

# importing necessary package
import pandas as pd

# Read csv file
df = pd.read_csv('emissions_data.csv')
df['abs_rel_error'] = abs(df['relative_error'])
# useful column
rel = df['abs_rel_error']

# get the required info
above150 = (rel > 150)
in150_100 = (rel > 100) & (rel <= 150)
in100_50 = (rel > 50) & (rel <= 100)
in50_20 = (rel > 20) & (rel <= 50)
in20_0 = (rel > 0) & (rel <= 20)

mean = round(rel.mean(), 3)
median = round(rel.median(), 3)
std = round(rel.std(), 3)

print("above 150% : ", above150.sum()) # 1
print("in between 100% and 150% : ", in150_100.sum()) # 38
print("in between 50% and 100% : ", in100_50.sum()) # 696
print("in between 20% and 50% : ", in50_20.sum()) # 3404
print("in between 0% and 20% : ", in20_0.sum()) # 2150
print("mean : ", mean) # 29.325
print("median : ", median) # 28.288
print("std : ", std) # 18.511

# According to the paper, " Majority of errors are concentrated below 10 Mt/yr or 50% with very few exceeding 100%." which we can again see is true

In [ ]:
""" This code gets us a few statistics about true emissions and predicted emissions. """

# necessary package
import pandas as pd

# read
df = pd.read_csv('emissions_data.csv')

true = df['True Emissions']
min_true = round(true.min(), 3)
max_true = round(true.max(), 3)
mean_true = round(true.mean(), 3)
std_true = round(true.std(), 3)
median_true = round(true.median(), 3)
range_true = round(max_true-min_true, 3)

pred = df['Pred_emiss']
min_pred = round(pred.min(), 3)
max_pred = round(pred.max(), 3)
mean_pred = round(pred.mean(), 3)
std_pred = round(pred.std(), 3)
median_pred = round(pred.median(), 3)
range_pred = round(max_pred-min_pred, 3)

print("True statistics : ")
print()
print("min : ", min_true) # 7.493
print("max : ", max_true) # 24.112
print("mean : ", mean_true) # 15.256
print("std : ", std_true) # 3.527
print("median : ", median_true) # 15.267
print("range : ", range_true) # 16.619
print()
print("Predicted statistics : ")
print()
print("min : ", min_pred) # 0.739
print("max : ", max_pred) # 34.250
print("mean : ", mean_pred) # 13.198
print("std : ", std_pred) # 5.202
print("median : ", median_pred) # 11.537
print("range : ", range_pred) # 33.511

# The mean true emission = 15.2 Mt/yr and mean pred emission = 13.1 Mt/yr which tantamounts to the fact that our model has closely reciprocated the results
# that was stated by Joffrey Dumont Le Brazidec et al.

Exploratory Data Analysis of our Train, Valid, Test sets.

In [ ]:
""" This cell is only meant for those who are interested in knowing what are the different variables present in our .nc files. """

# importing necessary package
from netCDF4 import Dataset

fn = 'curated/lip_train_dataset.nc' # filepath to .nc file
nc = Dataset(fn, 'r') # Reading the nc file
print("Variables in the NetCDF file:") 
print(nc.variables.keys()) # printing the different variables.
nc.close() # Closing the nc file to avoid corruption.

In [ ]:
""" This code is for us to plot the distribution of different variables that we have used so far to understand our data better. """

# importing necessary packages
import xarray as xr
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Declaration of filepaths and opening using xarray.
test_set = xr.open_dataset('curated/lip_test_dataset.nc')
valid_set = xr.open_dataset('curated/lip_valid_dataset.nc')
train_set = xr.open_dataset('curated/lip_train_dataset.nc')

datasets = [test_set, valid_set, train_set]
dataset_names = ['Test set', 'Validation set', 'Training set']
variables = ['point_source', 'time', 'xco2_noisy', 'no2_noisy', 'u', 'v', 'seg_pred_no2'] # Variables that are of interest to us. 

for ds, name in zip(datasets, dataset_names): # repeating it for all the sets
    for var in variables: # repeating it for all the variables
        flat_data = ds[var].values.reshape(ds[var].shape[0], -1)
        plt.figure(figsize = (12, 8))
        sns.histplot(flat_data.flatten(), bins = 80, kde = True) # Histogram plot using 80 bins
        plt.title(f"{name} set: {var} distribution")
        plt.xlabel(var)
        plt.ylabel('Frequency')
        plt.savefig(f"{name}_{var}_distribution.png")
        plt.show()    


In [ ]:
""" This code allows us to understand the temporal variations in our datasets."""

# importing necessary packages
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# Filepaths opened using xarray
test_set = xr.open_dataset('curated/lip_test_dataset.nc')
valid_set = xr.open_dataset('curated/lip_valid_dataset.nc')
train_set = xr.open_dataset('curated/lip_train_dataset.nc')

datasets = [test_set, valid_set, train_set]
dataset_names = ['Test set', 'Validation set', 'Training set']

for dataset, name in zip(datasets, dataset_names): # Repeating it for all the sets.
    time_values = dataset['time'] # extracts the time values
    time_index = pd.to_datetime(time_values.values) # Converting into a readable format.
    if name == 'Test set':
        hour_intervals = np.arange(0, 24, 1) # In test set, the temporal interval is 1 hour.
        xtick_labels = [f'{i}-{i+1}' for i in hour_intervals]
        
    else:
        hour_intervals = np.arange(0, 24, 2) # In train and valid set, the temporal interval is 2 hours.
        xtick_labels = [f'{i}-{i+2}' for i in hour_intervals]
    
    # Hourly Distribution plot
    plt.figure(figsize = (15, 8))
    hours = time_index.hour.value_counts().sort_index()
    bars = plt.bar(hours.index, hours, color = plt.cm.get_cmap('viridis', len(hour_intervals))(np.arange(len(hour_intervals)))) # plotting a bar plot 
    for bar in bars:
        yval = bar.get_height() # getting the exact value of the bar
        plt.text(bar.get_x() + bar.get_width()/2, yval, round(yval), va = 'bottom', ha = 'center', fontsize = 8) # Placing the value at the top of the bar.
    plt.title(f"{name} time distribution by hour")
    plt.xlabel('Hour of Day')
    plt.ylabel('Count')
    plt.xticks(hour_intervals, xtick_labels)
    plt.tight_layout()
    plt.savefig(f"{name}_hour_distribution.png")
    plt.show()
    
    # Weekly Distribution plot
    plt.figure(figsize = (10, 6))
    days = time_index.dayofweek.value_counts().sort_index()
    plt.bar(range(7), days, color = plt.cm.get_cmap('viridis', 7)(np.arange(7))) # plotting a bar plot
    plt.title(f"{name} time distribution by day of week")
    plt.xlabel('Day of Week')
    plt.ylabel('Count')
    plt.xticks(range(7), ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun'])
    for i, value in enumerate(days):
        plt.text(i, value + 0.5, str(value), ha = 'center', va = 'bottom', fontsize = 8, color = 'black') # Placing the value at the top of the bar.
    plt.tight_layout()
    plt.savefig(f"{name}_day_distribution.png")
    plt.show()
    
    # Monthly Distribution plot
    plt.figure(figsize = (10, 6))
    months = time_index.month.value_counts().sort_index()
    plt.bar(range(1, 13), months, color = plt.cm.get_cmap('viridis', 12)(np.arange(12))) # plotting a bar plot
    plt.title(f"{name} time distribution by month")
    plt.xlabel('Month')
    plt.ylabel('Count')
    plt.xticks(range(1, 13), ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])
    for i, value in enumerate(months):
        plt.text(i + 1, value + 0.5, str(value), ha = 'center', va = 'bottom', fontsize = 8, color = 'black') # Placing the value at the top of the bar.
    plt.tight_layout()
    plt.savefig(f"{name}_month_distribution.png")
    plt.show()

In [ ]:
""" This code helps us in identifying from which sources we got our datasets."""

# importing necessary package
import xarray as xr

# Filepaths opened using xarray
test_set = xr.open_dataset('Datasets/data_paper_inv_pp/lippendorf/test_dataset.nc')
valid_set = xr.open_dataset('Datasets/data_paper_inv_pp/lippendorf/valid_dataset.nc')
train_set = xr.open_dataset('Datasets/data_paper_inv_pp/lippendorf/train_dataset.nc')

# Only need point_source values
test_point_sources = test_set['point_source'].values
valid_point_sources = valid_set['point_source'].values
train_point_sources = train_set['point_source'].values

unique_test_point_sources = set(test_point_sources.tolist())
unique_valid_point_sources = set(valid_point_sources.tolist())
unique_train_point_sources = set(train_point_sources.tolist())

# Print these unique point sources
print("Unique point sources in Test set:")
print(unique_test_point_sources) # Only has data from Lippendorf
print()

# Have data from everywhere except Lippendorf
print("Unique point sources in Validation set:")
print(unique_valid_point_sources)
print()

print("Unique point sources in Training set:")
print(unique_train_point_sources)


In [ ]:
""" This code helps us in understanding temporal variations better and also look at emission data."""

# importing necessary packages
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

test_set = xr.open_dataset('curated/lip_test_dataset.nc')
valid_set = xr.open_dataset('curated/lip_valid_dataset.nc')
train_set = xr.open_dataset('curated/lip_train_dataset.nc')

#variables = ['xco2_noisy', 'no2_noisy', 'u', 'v', 'seg_pred_no2', 'emiss']
variables = ['emiss']
datasets = [test_set, valid_set, train_set]
dataset_names = ['Test set', 'Validation set', 'Training set']

for dataset, name in zip(datasets, dataset_names):
    #time_values = dataset['time']
    
    #print(f"{name} time distribution:")
    #print(f"Min time: {time_values.min().values}")
    #print(f"Max time: {time_values.max().values}")
    #time_diff_1_2 = (time_values[1] - time_values[0]).astype('timedelta64[ns]').values*(10**(-9))
    #time_diff_2_3 = (time_values[2] - time_values[1]).astype('timedelta64[ns]').values*(10**(-9)) 
    #time_diff_3_4 = (time_values[3] - time_values[2]).astype('timedelta64[ns]').values*(10**(-9))
    #time_range = (time_values.max() - time_values.min()).astype('timedelta64[ns]').values*(10**(-9))
    
    #print(f"Time difference between 1st and 2nd observation: {int(time_diff_1_2/3600)} h") # 1 h if test, 2 h if train or valid
    #print(f"Time difference between 2nd and 3rd observation: {int(time_diff_2_3/3600)} h") # 1 h if test, 2 h if train or valid
    #print(f"Time difference between 3rd and 4th observation: {int(time_diff_3_4/3600)} h") # 1 h if test, 2 h if train or valid
    #print()

    print(f"{name} set summary statistics:")
    for var in variables:
        if var == 'emiss': # only for emiss data
            emiss_data = dataset['emiss'].values
            modified_data = emiss_data * [0, 1, 0] # only considering the middle value
            emiss_data = np.round(modified_data, decimals = 3) # rounded off to 3 decimal places
            mean = np.mean(emiss_data, axis = None) # Calculating mean 
            max = np.max(emiss_data, axis = None) # Getting the max
            min = np.min(emiss_data, axis = None) # Getting the min
            std = np.std(emiss_data, axis = None) # Calculating the std
        
            print(f"mean : {mean}")
            print(f"max : {max}")
            print(f"min : {min}")
            print(f"std : {std}")
            print()
        
        #else: # for all variables except emiss
            #print(f"Variable {var}:")
            #print(f"mean : {dataset[var].mean().values}") # Calculating mean 
            #print(f"std dev : {dataset[var].std().values}") # Calculating the std
            #print(f"min : {dataset[var].min().values}") # Getting the min
            #print(f"max : {dataset[var].max().values}") # Getting the max
            #print()

In [ ]:
""" This code helps us in understanding the emiss distribution."""

# importing necessary packages
import xarray as xr
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Opened using xarray
test_set = xr.open_dataset('curated/lip_test_dataset.nc')
valid_set = xr.open_dataset('curated/lip_valid_dataset.nc')
train_set = xr.open_dataset('curated/lip_train_dataset.nc')
names = ['Test', 'Train', 'Val']

# Function Declaration
def process_emiss_data(name, dataset):
    emiss_data = dataset['emiss'].values
    modified_data = emiss_data * [0, 1, 0] # only considering the middle value
    emiss_data = np.round(modified_data, decimals = 3) # rounded off to 3 decimal places
    data = emiss_data.flatten()
    
    plt.figure(figsize = (10, 6))
    sns.histplot(data, kde = True, bins = 50, edgecolor = 'black') # Histogram plot to understand the emiss distribution
    plt.title('Histogram of Emiss Data')
    plt.xlabel('Value')
    plt.ylabel('Density')
    plt.grid(True)
    #plt.savefig(f"Figures/EDA/{name}/{name}_set_emiss_distribution.png")
    plt.show()

# Calling the function
process_emiss_data(names[0], test_set)
process_emiss_data(names[2], valid_set)
process_emiss_data(names[1], train_set)


In [ ]:
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np

# filepath declaration and opening with xarray
test_set = xr.open_dataset('curated/box_test_dataset.nc')
valid_set = xr.open_dataset('curated/box_valid_dataset.nc')
train_set = xr.open_dataset('curated/box_train_dataset.nc')
names = ['Test', 'Train', 'Val']
datasets = [test_set, train_set, valid_set]

# define the bins
bins = [(0, 2), (2, 5), (5, 10), (10, 15), (15, 20), (20, 25), (25, np.inf)]

for name, dataset in zip(names, datasets):
    print(f"Counts for {name} set:")
    emiss = dataset['emiss'].values
    mod = emiss[:, 1] 
    print(f"Shape of mod: {mod.shape}")
    total = len(mod)
    print(f"Total: {total}")
    
    # Calculate statistics
    min_val = np.min(mod)
    max_val = np.max(mod)
    range_val = max_val - min_val
    mean_val = np.mean(mod)
    median_val = np.median(mod)
    std_val = np.std(mod)
    
    print(f"Min: {min_val:.2f}")
    print(f"Max: {max_val:.2f}")
    print(f"Range: {range_val:.2f}")
    print(f"Mean: {mean_val:.2f}")
    print(f"Median: {median_val:.2f}")
    print(f"Standard Deviation: {std_val:.2f}")
    
    # loop through the bins and count the values
    for bin in bins:
        count = np.sum((mod >= bin[0]) & (mod < bin[1]))  # Use mod instead of data
        percentage = (count / total) * 100  # Calculate percentage
        if bin[1] == np.inf:
            print(f"{bin[0]} and above: {count} ({percentage:.2f}%)")
        else:
            print(f"{bin[0]}-{bin[1]}: {count} ({percentage:.2f}%)")
    
    '''# Plot histogram
    plt.hist(mod, bins=50, alpha=0.5, label=name)
    plt.xlabel('Emissivity')
    plt.ylabel('Frequency')
    plt.title(f'Histogram of Emissivity for {name} set')
    plt.savefig(f"Figures/EDA/{name}/{name}_set_emiss_distribution_actual.png")
    plt.show()'''
    
    print()

In [1]:
import numpy as np
from netCDF4 import Dataset as NetCDFDataset

class CustomDataset:
    
    def __init__(self, file_path, variable = None):
        self.ncfile = NetCDFDataset(file_path, 'r') # Reading the .nc file
        self.variable = variable # Variable in our comparison is no2_noisy
    
    def __len__(self):
        return self.ncfile.variables['xco2_noisy'].shape[0]
    
    def __getitem__(self, idx):

        
        emiss_read = self.ncfile.variables['emiss'][idx]
        
        emiss_arr = np.array(emiss_read.data).astype('float32')
        weights = np.array([0.0, 1.0, 0.0]) 
        weighted = np.round(np.sum(weights * emiss_arr), 3)
        outputs = np.array([weighted], dtype = np.float32)
        
        return outputs # returns our inputs (64, 64, 4), outputs (1,)

    def __del__(self):
        self.ncfile.close() # Closing the ncfile 
        
# Define paths to dataset files
test_set = 'curated/box_test_dataset.nc'
valid_set = 'curated/box_valid_dataset.nc'
train_set = 'curated/box_train_dataset.nc'

# Load custom datasets
train_dataset = CustomDataset(train_set, variable='no2_noisy')
valid_dataset = CustomDataset(valid_set, variable='no2_noisy')
test_dataset = CustomDataset(test_set, variable='no2_noisy')

# Initialize lists to store inputs and outputs

combine_outputs = []

# Combine inputs and outputs from all datasets
for dataset in [train_dataset, valid_dataset, test_dataset]:
    for outputs in dataset:
        
        combine_outputs.append(outputs)


combine_outputs = np.array(combine_outputs)  # Shape: (36049, 1)

# Shuffle dataset
indices = np.arange(combine_outputs.shape[0])
np.random.shuffle(indices)
combine_outputs = combine_outputs[indices]

# Split the dataset into training, validation, and test sets
train_size = int(0.65 * combine_outputs.shape[0])
valid_size = int(0.15 * combine_outputs.shape[0])
test_size = combine_outputs.shape[0] - train_size - valid_size

train_outputs, val_outputs, test_outputs = np.split(combine_outputs, [train_size, train_size + valid_size])

# Clean up unnecessary variables
del combine_outputs, train_dataset, valid_dataset, test_dataset, indices

# Function to calculate statistics
def calculate_statistics(data):
    min_val = np.min(data)
    max_val = np.max(data)
    mean_val = np.mean(data)
    median_val = np.median(data)
    std_val = np.std(data)
    return min_val, max_val, mean_val, median_val, std_val

# Calculate and print statistics for each dataset
datasets = [train_outputs, val_outputs, test_outputs]
names = ['Train', 'Validation', 'Test']

for name, dataset in zip(names, datasets):
    # Flatten dataset for analysis
    mod = dataset.flatten()
    
    # Calculate statistics
    min_val, max_val, mean_val, median_val, std_val = calculate_statistics(mod)
    
    print(f"\nStatistics for {name} set:")
    print(f"Min: {min_val:.2f}")
    print(f"Max: {max_val:.2f}")
    print(f"Mean: {mean_val:.2f}")
    print(f"Median: {median_val:.2f}")
    print(f"Standard Deviation: {std_val:.2f}")
    
    # Define bins for histogram analysis
    bins = [(0, 2), (2, 5), (5, 10), (10, 15), (15, 20), (20, 25), (25, np.inf)]
    
    # Histogram analysis
    total = len(mod)
    for bin_range in bins:
        count = np.sum((mod >= bin_range[0]) & (mod < bin_range[1]))
        percentage = (count / total) * 100
        if bin_range[1] == np.inf:
            print(f"{bin_range[0]} and above: {count} ({percentage:.2f}%)")
        else:
            print(f"{bin_range[0]}-{bin_range[1]}: {count} ({percentage:.2f}%)")



Statistics for Train set:
Min: 2.88
Max: 52.66
Mean: 13.99
Median: 10.48
Standard Deviation: 9.12
0-2: 0 (0.00%)
2-5: 1189 (5.07%)
5-10: 9909 (42.29%)
10-15: 4131 (17.63%)
15-20: 3107 (13.26%)
20-25: 2298 (9.81%)
25 and above: 2797 (11.94%)

Statistics for Validation set:
Min: 2.95
Max: 52.66
Mean: 14.01
Median: 10.69
Standard Deviation: 9.00
0-2: 0 (0.00%)
2-5: 285 (5.27%)
5-10: 2218 (41.02%)
10-15: 1008 (18.64%)
15-20: 691 (12.78%)
20-25: 560 (10.36%)
25 and above: 645 (11.93%)

Statistics for Test set:
Min: 2.86
Max: 52.66
Mean: 14.04
Median: 10.50
Standard Deviation: 9.20
0-2: 0 (0.00%)
2-5: 394 (5.46%)
5-10: 3037 (42.12%)
10-15: 1270 (17.61%)
15-20: 952 (13.20%)
20-25: 661 (9.17%)
25 and above: 897 (12.44%)


In [ ]:
""" This code helps us in plotting a few random plots of xco2, no2, seg images."""

# importing necessary packages
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np

# filepath declaration and opening with xarray
test_set = xr.open_dataset('curated/lip_test_dataset.nc')
valid_set = xr.open_dataset('curated/lip_valid_dataset.nc')
train_set = xr.open_dataset('curated/lip_train_dataset.nc')
names = ['Test', 'Train', 'Val']

# Function declaration
def plot(name, dataset):
    
    # Get the values from the variables that are of interest
    no2_noisy = dataset['no2_noisy'].values
    xco2_noisy = dataset['xco2_noisy'].values
    u = dataset['u'].values
    v = dataset['v'].values
    seg_pred_no2 = dataset['seg_pred_no2'].values
    num_images = 4 # total number of images we want to plot for each.
    
    # PLotting NO2 images
    fig, axs = plt.subplots(1, num_images, figsize = (12, 8))
    for i in range(num_images):
        idx = np.random.randint(0, len(no2_noisy)) # generates a random number in between 0 and length of the column-1
        img = axs[i].imshow(no2_noisy[idx], cmap = 'viridis')
        axs[i].set_title(f'no2_noisy Image {idx}') 
        axs[i].axis('off')
        cbar = fig.colorbar(img, ax = axs[i], orientation = 'vertical', fraction = 0.05)
        cbar.set_label('Intensity', rotation = 270, labelpad = 25)
    plt.tight_layout()
    plt.savefig(f"Figures/EDA/{name}/{name}_set_no2.png")
    plt.show()

    # PLotting XCO2 images 
    fig, axs = plt.subplots(1, num_images, figsize = (12, 8))
    for i in range(num_images):
        idx = np.random.randint(0, len(xco2_noisy)) # generates a random number in between 0 and length of the column-1
        img = axs[i].imshow(xco2_noisy[idx], cmap = 'viridis')
        axs[i].set_title(f'xco2_noisy Image {idx}')
        axs[i].axis('off')
        cbar = fig.colorbar(img, ax = axs[i], orientation = 'vertical', fraction = 0.05)
        cbar.set_label('Intensity', rotation = 270, labelpad = 15)
    plt.tight_layout()
    plt.savefig(f"Figures/EDA/{name}/{name}_set_xco2.png")
    plt.show()

    # PLotting seg images
    fig, axs = plt.subplots(1, num_images, figsize = (12, 8))
    for i in range(num_images):
        idx = np.random.randint(0, len(seg_pred_no2)) # generates a random number in between 0 and length of the column-1
        img = axs[i].imshow(seg_pred_no2[idx], cmap = 'viridis')
        axs[i].set_title(f'seg_pred_no2 Image {idx}')
        axs[i].axis('off')
        cbar = fig.colorbar(img, ax = axs[i], orientation = 'vertical', fraction = 0.05)
        cbar.set_label('Intensity', rotation = 270, labelpad = 15)
    plt.tight_layout()
    plt.savefig(f"Figures/EDA/{name}/{name}_set_seg.png")
    plt.show()

# Calling the function
plot(names[0], test_set)
plot(names[2], valid_set)
plot(names[1], train_set)


Segmentation

In [ ]:
""" Let us understand our input data and target data for segmentation work. """

# necessary packages
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np

# filepath declaration and opening with xarray
test_set = xr.open_dataset('curated/lip_test_dataset.nc')
valid_set = xr.open_dataset('curated/lip_valid_dataset.nc')
train_set = xr.open_dataset('curated/lip_train_dataset.nc')
names = ['Test', 'Train', 'Val']

# Function declaration
def plot(name, dataset):
    
    # Get the random value from the variables
    idx = np.random.randint(0, len(dataset['no2']))
    
    no2 = dataset['no2'][idx].values
    plume = dataset['no2_plume'][idx].values
    bool_perf_seg = dataset['plume'][idx].values
    seg_pred_no2 = dataset['seg_pred_no2'][idx].values
    w_perf_seg = dataset['w_perf_seg'][idx].values

    fig, axs = plt.subplots(1, 5, figsize = (18, 6))

    axs[0].imshow(no2, cmap = 'viridis')
    axs[0].set_title(f'{name} - NO2')
    axs[1].imshow(plume, cmap = 'viridis')
    axs[1].set_title(f'{name} - plume')
    #fig.colorbar(im, ax = axs[1])
    axs[2].imshow(bool_perf_seg, cmap = 'viridis')
    axs[2].set_title(f'{name} - Bool Perf Seg')
    axs[3].imshow(seg_pred_no2, cmap = 'viridis')
    axs[3].set_title(f'{name} - Seg Pred NO2')
    axs[4].imshow(w_perf_seg, cmap = 'viridis')
    axs[4].set_title(f'{name} - W Perf Seg')
    

    #plt.tight_layout()
    #plt.savefig(f'Figures/EDA/{name}/comp.png')
    plt.show()

# Function calling
for name, dataset in zip(names, [test_set, train_set, valid_set]):
    plot(name, dataset)


In [ ]:
""" Segmentation Code for NO2 data """

"""
Importing different packages that we will use.

"""
import h5py
import os
import numpy as np
from netCDF4 import Dataset as NetCDFDataset
from tensorflow.keras.layers import Conv2D, Dropout, BatchNormalization, MaxPool2D, Input, Concatenate, Lambda, Activation, UpSampling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tqdm import tqdm
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow import keras

"""
We define our Custom Dataset which will read the .nc files (train, valid, test) and extract the no2, plume data from the .nc files.
Note to the reader : For our comparision, we are going to consider Lippendorf location.

"""
class CustomDataset:
    
    def __init__(self, file_path):
        self.ncfile = NetCDFDataset(file_path, 'r') # Reading the .nc file
    
    def __len__(self):
        return self.ncfile.variables['no2'].shape[0] # The total number of images or data we will have which is different for train (25152), valid (4608), test (6289)

    # To preprocess plume data as mentioned by "Segmentation of XCO2 images with deep learning: application to synthetic plumes from cities and power plants" by Joffrey Dumont Le Brazidec et al.
    def calculate_weighted_plume(self, plume, min_w, max_w, curve = "linear", param_curve = 1):
        y_min = 0.05
        y_max = np.quantile(plume, q = 0.99)
    
        weight_min = min_w
        weight_max = max_w
    
        y_data = np.zeros_like(plume, dtype = np.float32)
    
        if curve == "linear":
            y_data[plume <= y_min] = 0
            y_data[(plume > y_min) & (plume < y_max)] = weight_min + ((weight_max - weight_min) / (y_max - y_min)) * (plume[(plume > y_min) & (plume < y_max)] - y_min)
            y_data[plume >= y_max] = weight_max
    
        return y_data
        
    def __getitem__(self, idx):

        # Reading the necessary variables (no2, w_perf_seg, seg_pred_no2)
        no2_read = self.ncfile.variables['no2'][idx] # Unit : molec/cm^2, Shape : 64, 64
        plume_read = self.ncfile.variables['plume'][idx] # Unit : ppmv, Shape : 64, 64
        
        # Converting into numpy arrays
        no2_arr = np.array(no2_read.data).astype('float32')
        plume_arr = np.array(plume_read.data).astype('float32')
        
        # Normalization
        inputs = (no2_arr - no2_arr.mean(axis = (0, 1), keepdims = True)) / np.where(no2_arr.std(axis = (0, 1), keepdims = True) == 0, 1, no2_arr.std(axis = (0, 1), keepdims = True))
        truths = self.calculate_weighted_plume(plume_arr, min_w = 0.01, max_w = 4.0)
        inputs = inputs.reshape((64, 64, 1))
        truths = truths.reshape((64, 64, 1))
        
        return inputs, truths

    def __del__(self):
        self.ncfile.close()

""" Our Unet model builder function 
Note to the reader :  The paper "Segmentation of XCO2 images with deep learning: application to synthetic plumes from cities and power plants" by Joffrey Dumont Le Brazidec et al.
used EfficientNetB0 as its encoder in the Unet architecture which resulted 10 million params. The code is taken from the paper (Github). """

# Two-Dimensional Convolutional operation with Batch Normalization
def Conv2dBn(filters, kernel_size, strides = (1, 1), padding = 'valid', data_format = None, dilation_rate = (1, 1), activation = None, 
             kernel_initializer = 'glorot_uniform', bias_initializer = 'zeros', kernel_regularizer = None, bias_regularizer = None, 
             activity_regularizer = None, kernel_constraint = None, bias_constraint = None, use_batchnorm = False, **kwargs):
    def wrapper(input_tensor):
        x = Conv2D(
            filters = filters,
            kernel_size = kernel_size,
            strides = strides,
            padding = padding,
            data_format = data_format,
            dilation_rate = dilation_rate,
            activation = None,
            use_bias = not (use_batchnorm),
            kernel_initializer = kernel_initializer,
            bias_initializer = bias_initializer,
            kernel_regularizer = kernel_regularizer,
            bias_regularizer = bias_regularizer,
            activity_regularizer = activity_regularizer,
            kernel_constraint = kernel_constraint,
            bias_constraint = bias_constraint,
            )(input_tensor)

        if use_batchnorm:
            x = BatchNormalization()(x)

        if activation:
            x = Activation(activation)(x)

        return x

    return wrapper

# Convolution operation that calls the previous defined function with activation as ReLU.
def Conv3x3BnReLU(filters, use_batchnorm, name = None):
    def wrapper(input_tensor):
        return Conv2dBn(
            filters,
            kernel_size = 3,
            activation = 'relu',
            kernel_initializer = 'he_uniform',
            padding = 'same',
            use_batchnorm = use_batchnorm,
            name = name,
        )(input_tensor)

    return wrapper

# For the Decoder part of Unet
def DecoderUpsamplingX2Block(filters, stage, use_batchnorm = False):
    up_name = 'decoder_stage{}_upsampling'.format(stage)
    conv1_name = 'decoder_stage{}a'.format(stage)
    conv2_name = 'decoder_stage{}b'.format(stage)
    concat_name = 'decoder_stage{}_concat'.format(stage)

    def wrapper(input_tensor, skip = None):
        x = UpSampling2D(size = 2, name = up_name)(input_tensor)

        if skip is not None:
            x = Concatenate(axis = 3, name = concat_name)([x, skip])

        x = Conv3x3BnReLU(filters, use_batchnorm, name = conv1_name)(x)
        x = Conv3x3BnReLU(filters, use_batchnorm, name = conv2_name)(x)
        return x

    return wrapper

def build_unet(backbone, decoder_block, skip_connection_layers, 
               decoder_filters = (160, 80, 40, 20, 10), n_upsample_blocks = 5, 
               classes = 1, activation = 'sigmoid', use_batchnorm = True):
    input_ = backbone.input
    x = backbone.output

    x = Lambda(lambda x: x)(x)

    # extract skip connections
    skips = [backbone.get_layer(name = i).output if isinstance(i, str) 
             else backbone.get_layer(index = i).output for i in skip_connection_layers]

    # building decoder blocks
    for i in range(n_upsample_blocks):
        if i < len(skips):
            skip = skips[i]
        else:
            skip = None

        x = decoder_block(decoder_filters[i], stage = i, use_batchnorm = use_batchnorm)(x, skip)

    x = Conv2D(
        filters = classes,
        kernel_size = (3, 3),
        padding = 'same',
        use_bias = True,
        kernel_initializer = 'glorot_uniform',
        name = 'final_conv',
    )(x)
    x = Activation(activation, name = activation)(x)

    model = Model(input_, x)

    return model

def Unet_model(input_shape, num_classes, backbone_trainable = True):
    inputs = Input(shape = input_shape)
    #x = Conv2D(3, (3, 3), activation = 'relu', padding = 'same')(inputs)
    backbone = keras.applications.EfficientNetB0(include_top = False, input_tensor = inputs, weights = None, drop_connect_rate = 0.2)
    backbone.trainable = backbone_trainable
    model = build_unet(
        backbone = backbone,
        decoder_block = DecoderUpsamplingX2Block,
        skip_connection_layers = ['block6a_expand_activation', 'block4a_expand_activation',
                                'block3a_expand_activation', 'block2a_expand_activation'],
        decoder_filters = (256, 128, 64, 32, 16),
        classes = num_classes,
        activation = 'sigmoid',
        use_batchnorm = True,
    )
    return model

# Custom Loss function adapted from "Segmentation of XCO2 images with deep learning: application to synthetic plumes from cities and power plants" by Joffrey Dumont Le Brazidec et al.
def weighted_loss():
    def loss(y_true, y_pred):
        y_bin_true = tf.cast(y_true > 0, y_true.dtype)
        loss_val = tf.keras.losses.binary_crossentropy(y_bin_true, y_pred)
        weights = tf.where(y_true > 0, y_true, 1.0)
        loss_val = tf.squeeze(weights, axis = -1) * loss_val
        return K.mean(loss_val)
    return loss

# Creating and checking if saved_models folder exists or not. If not, then create a folder where we will store our weights.h5 file.
checkpoint_dir = './saved_models/' 
os.makedirs(checkpoint_dir, exist_ok = True)

best_model_filepath = os.path.join(checkpoint_dir, 'seg_weights.h5')
# Callbacks can be of best weights or last weights. We can do either one. We went with saving the weights after every epoch. 
best_model_checkpoint_callback = ModelCheckpoint(filepath = best_model_filepath, save_weights_only = True, verbose = 1)

# Filepaths for Train, Valid, Test. From the paper we know that for Train, Valid we will choose data points from everywhere except the location in which
# we will test our model's performance. Location = Lippendorf
test_set = 'curated/lip_train_dataset.nc'
valid_set = 'curated/lip_valid_dataset.nc'
train_set = 'curated/lip_train_dataset.nc'

# Here, we have generated our custom dataset.
train_dataset = CustomDataset(train_set)
valid_dataset = CustomDataset(valid_set)
test_dataset = CustomDataset(test_set)

# Applying data augmentation to Training data.
data_augmentation = ImageDataGenerator(rotation_range = 180, width_shift_range = 0.2, height_shift_range = 0.2, shear_range = 45, zoom_range = 0.2, horizontal_flip = True, vertical_flip = True, fill_mode = 'nearest')

# To store the inputs and outputs from our dataset to feed it to the model during training, validation and inference.
train_inputs = []
train_truths = []
val_inputs = []
val_truths = []
test_inputs = []
test_truths = []

for inputs, truths in train_dataset:
    train_inputs.extend([inputs])
    train_truths.extend([truths])
for inputs, truths in valid_dataset:
    val_inputs.extend([inputs])
    val_truths.extend([truths])
for inputs, truths in test_dataset:
    test_inputs.extend([inputs])
    test_truths.extend([truths])

# Converting into numpy arrays
train_inputs = np.array(train_inputs) # train_inputs shape : (25152, 64, 64, 1) 
train_truths = np.array(train_truths) # train_truths shape : (25152, 64, 64, 1)
val_inputs = np.array(val_inputs) # val_inputs shape : (4608, 64, 64, 1) 
val_truths = np.array(val_truths) # val_truths shape : (4608, 64, 64, 1)
test_inputs = np.array(test_inputs) # test_inputs shape : (6289, 64, 64, 1)
test_truths = np.array(test_truths) # test_truths shape : (6289, 64, 64, 1)

# Model Declaration
model = Unet_model(input_shape = (64, 64, 1), num_classes = 1)

#model.load_weights('saved_models/seg_weights.h5') # If we have already stored the weights, then uncomment this line to load.

optimizer = Adam(learning_rate = 1e-3) # Taking Adam optimizer with lr = 1e-3

# We will update our lr if we reach a plateau. For this we use ReduceLROnPlateau which will monitor val_loss, wait for 20 epochs before changing lr and measures 
# min_delta threshold, reduces lr by 0.5 and lower bound on lr is 5e-5.
learning_rate_monitor_callback = ReduceLROnPlateau(monitor = 'val_loss', factor = 0.5, patience = 20, verbose = 0, min_delta = 5e-3, cooldown = 0, min_lr = 5e-5)

# Loss function for this task is Custom Weighted BCE.
model.compile(optimizer = optimizer, loss = weighted_loss(), metrics = ['accuracy'])

# Generate the summary of the model. Around 5 million trainable paramaters are there.
model.summary()

# Total epochs = 500, batch size for training set = 32, steps will be 25152/32 = 786, validation will not have any augmentation with batch size of 32 and validating
# on the entire validation set.
history = model.fit(data_augmentation.flow(train_inputs, train_truths, batch_size = 32, shuffle = True), epochs = 500, steps_per_epoch = len(train_inputs)//32,
validation_data = (val_inputs, val_truths), validation_batch_size = 32, validation_steps = None, callbacks = [learning_rate_monitor_callback, best_model_checkpoint_callback])

# To test the model's performance on test set.
test_loss, test_accuracy = model.evaluate(test_inputs, test_truths, batch_size = None)
print(f'Test Loss: {test_loss:.4f}') # 0.1471
print(f'Test Accuracy: {test_accuracy:.4f}') # 0.7907

# Plotting loss and accuracy curves
plt.plot(history.history['loss'], label = 'Training Loss')
plt.plot(history.history['val_loss'], label = 'Validation Loss')
plt.plot(history.history['accuracy'], label = 'Training Accuracy')
plt.plot(history.history['val_accuracy'], label = 'Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Loss/Accuracy')
plt.legend()
#plt.savefig('Figures/Seg/check_loss.png')
plt.show()


In [ ]:
""" Check for model performance """

import h5py
import os
import numpy as np
from netCDF4 import Dataset as NetCDFDataset
from tensorflow.keras.layers import Conv2D, Dropout, BatchNormalization, MaxPool2D, Input, Concatenate, Lambda, Activation, UpSampling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow import keras

class CustomDataset:
    
    def __init__(self, file_path):
        self.ncfile = NetCDFDataset(file_path, 'r') # Reading the .nc file
    
    def __len__(self):
        return self.ncfile.variables['no2'].shape[0] # The total number of images or data we will have which is different for train (25152), valid (4608), test (6289)

    # To preprocess plume data as mentioned by "Segmentation of XCO2 images with deep learning: application to synthetic plumes from cities and power plants" by Joffrey Dumont Le Brazidec et al.
    def calculate_weighted_plume(self, plume, min_w, max_w, curve = "linear", param_curve = 1):
        y_min = 0.05
        y_max = np.quantile(plume, q = 0.99)
    
        weight_min = min_w
        weight_max = max_w
    
        y_data = np.zeros_like(plume, dtype = np.float32)
    
        if curve == "linear":
            y_data[plume <= y_min] = 0
            y_data[(plume > y_min) & (plume < y_max)] = weight_min + ((weight_max - weight_min) / (y_max - y_min)) * (plume[(plume > y_min) & (plume < y_max)] - y_min)
            y_data[plume >= y_max] = weight_max
    
        return y_data
        
    def __getitem__(self, idx):

        # Reading the necessary variables (no2, w_perf_seg, seg_pred_no2)
        no2_read = self.ncfile.variables['no2'][idx] # Unit : molec/cm^2, Shape : 64, 64
        plume_read = self.ncfile.variables['plume'][idx] # Unit : ppmv, Shape : 64, 64
        seg_read = self.ncfile.variables['seg_pred_no2'][idx] # Unit : bool, Shape :  64, 64
        
        # Converting into numpy arrays
        no2_arr = np.array(no2_read.data).astype('float32')
        plume_arr = np.array(plume_read.data).astype('float32')
        seg_arr = np.array(seg_read.data).astype('float32')
        
        # Normalization
        inputs = (no2_arr - no2_arr.mean(axis = (0, 1), keepdims = True)) / np.where(no2_arr.std(axis = (0, 1), keepdims = True) == 0, 1, no2_arr.std(axis = (0, 1), keepdims = True))
        seg = (seg_arr - seg_arr.mean(axis = (0, 1), keepdims = True))/np.where(seg_arr.std(axis = (0, 1), keepdims = True) == 0, 1, seg_arr.std(axis = (0, 1), keepdims = True))
        truths = self.calculate_weighted_plume(plume_arr, min_w = 0.01, max_w = 4.0)
        inputs = inputs.reshape((64, 64, 1))
        truths = truths.reshape((64, 64, 1))
        seg = seg.reshape((64, 64, 1))
        
        return inputs, truths, seg

    def __del__(self):
        self.ncfile.close()

# Two-Dimensional Convolutional operation with Batch Normalization
def Conv2dBn(filters, kernel_size, strides = (1, 1), padding = 'valid', data_format = None, dilation_rate = (1, 1), activation = None, 
             kernel_initializer = 'glorot_uniform', bias_initializer = 'zeros', kernel_regularizer = None, bias_regularizer = None, 
             activity_regularizer = None, kernel_constraint = None, bias_constraint = None, use_batchnorm = False, **kwargs):
    def wrapper(input_tensor):
        x = Conv2D(
            filters = filters,
            kernel_size = kernel_size,
            strides = strides,
            padding = padding,
            data_format = data_format,
            dilation_rate = dilation_rate,
            activation = None,
            use_bias = not (use_batchnorm),
            kernel_initializer = kernel_initializer,
            bias_initializer = bias_initializer,
            kernel_regularizer = kernel_regularizer,
            bias_regularizer = bias_regularizer,
            activity_regularizer = activity_regularizer,
            kernel_constraint = kernel_constraint,
            bias_constraint = bias_constraint,
            )(input_tensor)

        if use_batchnorm:
            x = BatchNormalization()(x)

        if activation:
            x = Activation(activation)(x)

        return x

    return wrapper

# Convolution operation that calls the previous defined function with activation as ReLU.
def Conv3x3BnReLU(filters, use_batchnorm, name = None):
    def wrapper(input_tensor):
        return Conv2dBn(
            filters,
            kernel_size = 3,
            activation = 'relu',
            kernel_initializer = 'he_uniform',
            padding = 'same',
            use_batchnorm = use_batchnorm,
            name = name,
        )(input_tensor)

    return wrapper

# For the Decoder part of Unet
def DecoderUpsamplingX2Block(filters, stage, use_batchnorm = False):
    up_name = 'decoder_stage{}_upsampling'.format(stage)
    conv1_name = 'decoder_stage{}a'.format(stage)
    conv2_name = 'decoder_stage{}b'.format(stage)
    concat_name = 'decoder_stage{}_concat'.format(stage)

    def wrapper(input_tensor, skip = None):
        x = UpSampling2D(size = 2, name = up_name)(input_tensor)

        if skip is not None:
            x = Concatenate(axis = 3, name = concat_name)([x, skip])

        x = Conv3x3BnReLU(filters, use_batchnorm, name = conv1_name)(x)
        x = Conv3x3BnReLU(filters, use_batchnorm, name = conv2_name)(x)
        return x

    return wrapper

def build_unet(backbone, decoder_block, skip_connection_layers, 
               decoder_filters = (160, 80, 40, 20, 10), n_upsample_blocks = 5, 
               classes = 1, activation = 'sigmoid', use_batchnorm = True):
    input_ = backbone.input
    x = backbone.output

    x = Lambda(lambda x: x)(x)

    # extract skip connections
    skips = [backbone.get_layer(name = i).output if isinstance(i, str) 
             else backbone.get_layer(index = i).output for i in skip_connection_layers]

    # building decoder blocks
    for i in range(n_upsample_blocks):
        if i < len(skips):
            skip = skips[i]
        else:
            skip = None

        x = decoder_block(decoder_filters[i], stage = i, use_batchnorm = use_batchnorm)(x, skip)

    x = Conv2D(
        filters = classes,
        kernel_size = (3, 3),
        padding = 'same',
        use_bias = True,
        kernel_initializer = 'glorot_uniform',
        name = 'final_conv',
    )(x)
    x = Activation(activation, name = activation)(x)

    model = Model(input_, x)

    return model

def Unet_model(input_shape, num_classes, backbone_trainable = True):
    inputs = Input(shape = input_shape)
    #x = Conv2D(3, (3, 3), activation = 'relu', padding = 'same')(inputs)
    backbone = keras.applications.EfficientNetB0(include_top = False, input_tensor = inputs, weights = None, drop_connect_rate = 0.2)
    backbone.trainable = backbone_trainable
    model = build_unet(
        backbone = backbone,
        decoder_block = DecoderUpsamplingX2Block,
        skip_connection_layers = ['block6a_expand_activation', 'block4a_expand_activation',
                                'block3a_expand_activation', 'block2a_expand_activation'],
        decoder_filters = (256, 128, 64, 32, 16),
        classes = num_classes,
        activation = 'sigmoid',
        use_batchnorm = True,
    )
    return model

def weighted_loss():
    def loss(y_true, y_pred):
        y_bin_true = tf.cast(y_true > 0, y_true.dtype)
        loss_val = tf.keras.losses.binary_crossentropy(y_bin_true, y_pred)
        weights = tf.where(y_true > 0, y_true, 1.0)
        loss_val = tf.squeeze(weights, axis = -1) * loss_val
        return K.mean(loss_val)
    return loss
    
test_set = 'curated/lip_test_dataset.nc'
test_dataset = CustomDataset(test_set)

test_inputs = []
test_truths = []
test_seg = []
for inputs, truths, seg in test_dataset:
    test_inputs.extend([inputs])
    test_truths.extend([truths])
    test_seg.extend([seg])
test_inputs = np.array(test_inputs) # test_inputs shape : (6289, 64, 64, 1)
test_truths = np.array(test_truths) # test_truths shape : (6289, 64, 64, 1)
test_seg = np.array(test_seg) # test_seg shape :  (6289, 64, 64, 1)

# Model Declaration
model = Unet_model(input_shape = (64, 64, 1), num_classes = 1)
model.load_weights('saved_models/seg_weights.h5')
optimizer = Adam(learning_rate = 1e-3)
model.compile(optimizer = optimizer, loss = weighted_loss(), metrics = ['accuracy'])
test_loss, test_accuracy = model.evaluate(test_inputs, test_truths, batch_size = None)
print(f'Test Loss: {test_loss:.4f}') # 0.1471
print(f'Test Accuracy: {test_accuracy:.4f}') # 0.7907

# Plots
test_pred = model.predict(test_inputs)
#test_pred = test_pred*4
idx = np.random.choice(test_inputs.shape[0], 25, replace = False)
for i in idx:
    plt.figure(figsize = (12, 4))
    plt.subplot(1, 4, 1)
    im1 = plt.imshow(test_inputs[i, :, :, 0], cmap = 'viridis')
    plt.title('Actual Image')
    plt.colorbar(im1, ax = plt.gca(), fraction = 0.046, pad = 0.04)
    plt.subplot(1, 4, 2)
    im2 = plt.imshow(test_truths[i, :, :, 0], cmap = 'viridis')
    plt.title('Mask used')
    plt.colorbar(im2, ax = plt.gca(), fraction = 0.046, pad = 0.04)
    plt.subplot(1, 4, 3)
    im3 = plt.imshow(test_pred[i, :, :, 0], cmap = 'viridis')
    plt.title('Predicted Mask')
    plt.colorbar(im3, ax = plt.gca(), fraction = 0.046, pad = 0.04)
    plt.subplot(1, 4, 4)
    im4 = plt.imshow(test_seg[i, :, :, 0], cmap = 'viridis')
    plt.title('Mask given')
    plt.colorbar(im4, ax = plt.gca(), fraction = 0.046, pad = 0.04)
    #plt.savefig(f'Figures/Seg/test_seg_ex_{i}.png')
    plt.show()

U-Net Regression Model approach

In [ ]:
""" This marks the beginning of our novel approach that is using U-Net for regression task. """
"""
Importing different packages that we will use.

"""
import h5py
import os
import numpy as np
from netCDF4 import Dataset as NetCDFDataset
from tensorflow.keras.layers import Input, Conv2D, Dropout, BatchNormalization, MaxPool2D, Concatenate, Conv2DTranspose, Flatten, Dense, UpSampling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tqdm import tqdm
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
import pandas as pd
import tensorflow as tf

"""
We define our Custom Dataset which will read the .nc files (train, valid, test) and extract the XCO2, u, v, NO2 data from the .nc files.
Note to the reader : For our comparision, we are going to consider Lippendorf location and as additional input we will consider NO2 images.

"""
class CustomDataset:
    
    def __init__(self, file_path, variable = None):
        self.ncfile = NetCDFDataset(file_path, 'r') # Reading the .nc file
        self.variable = variable # Variable in our comparison is no2_noisy
    
    def __len__(self):
        return self.ncfile.variables['xco2_noisy'].shape[0] # The total number of images or data we will have which is different for train (25152), valid (4608), test (6289)
    
    def __getitem__(self, idx):

        # Reading the necessary variables (xco2_noisy, u, v)
        xco2_read = self.ncfile.variables['xco2_noisy'][idx] # Unit : ppmv (parts per million by volume), Shape : 64, 64
        u_read = self.ncfile.variables['u'][idx] # Unit : m/s, Shape : 64, 64
        v_read = self.ncfile.variables['v'][idx] # Unit : m/s, Shape : 64, 64
        emiss_read = self.ncfile.variables['emiss'][idx] # emiss is the true output data that we need to train our model. Unit : Mt/yr (Million tonnes/ year)
                                                         # Shape : 3,
        # Converting into numpy arrays with data type float32
        xco2_arr = np.array(xco2_read.data).astype('float32') 
        u_arr = np.array(u_read.data).astype('float32')
        v_arr = np.array(v_read.data).astype('float32')
        emiss_arr = np.array(emiss_read.data).astype('float32')

        #noise = GaussianNoise(stddev = 0.7) # Addition of Gaussian Noise with std of 0.7
        #xco2 = noise(xco2_arr).numpy() 
        if self.variable:
            var_read = self.ncfile.variables[self.variable][idx] # Unit : molec/cm^2, Shape : 64, 64
            var_arr = np.array(var_read.data).astype('float32') # Converting into numpy array after reading the variable
            #var_noise = noise(var_tensor).numpy()
            inputs = np.stack([xco2_arr, u_arr, v_arr, var_arr], axis = -1) # Stacking the inputs to get the desired input shape = (64, 64, 4) where 64, 64
                                                                            # represents height, width and 4 represents the total number of features. Each pixel
                                                                            # has a spatial resolution of 2km, i.e. 1*1 in pixel refers to an area of 2km*2km.
        else: 
            inputs = np.stack([xco2_arr, u_arr, v_arr], axis = -1) # Stacking the inputs to get the shape = (64, 64, 3) in case when variable is None.
        
        mean = inputs.mean(axis = (0, 1), keepdims = True) # Calculating the mean of the 4 features separately and keeping the same shape as that of inputs.
        
        std = inputs.std(axis = (0, 1), keepdims = True) # Calculating the std of the 4 features separately and keeping the same shape as that of inputs.
        
        std[std==0] = 1 # If std becomes 0, we change it 1 to avoid division by 0. Multiple other methods can also be employed such as choosing an 
                        # appropriate epsilon such as 1e-5, 1e-6 or 1e-7
        
        inputs = (inputs - mean)/std # Normalization of the inputs separately for each feature.
        
        # emiss data is of shape (3,), from which we will only consider the middle value for our comparison. Hence, we multiply weights with emiss to
        # get the middle value and round it off to 3 decimal places.
        weights = np.array([0.0, 1.0, 0.0]) 
        weighted = np.round(np.sum(weights * emiss_arr), 3)
        outputs = np.array([weighted], dtype = np.float32)
        
        return inputs, outputs # returns our inputs (64, 64, 4), outputs (1,)

    def __del__(self):
        self.ncfile.close() # Closing the ncfile to avoid corruption of the file.

""" The U-Net regression Model """

def unetreg(input_shape, dropout_rate = 0.2):
    inputs = Input(input_shape) # If we use no2 then shape : (64, 64, 4), if we don't use no2 then shape : (64, 64, 3)
    
    # Encoder (downsampling path)
    c1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_uniform')(inputs) # Shape : (64, 64, 64)
    c1 = BatchNormalization()(c1)
    c1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_uniform')(c1) # Shape : (64, 64, 64)
    c1 = BatchNormalization()(c1)
    d1 = Dropout(dropout_rate)(c1) 
    p1 = MaxPool2D(pool_size = (2, 2))(d1) # Shape : (32, 32, 64) 
    
    c2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_uniform')(p1) # Shape : (32, 32, 128)
    c2 = BatchNormalization()(c2)
    c2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_uniform')(c2) # Shape : (32, 32, 128)
    c2 = BatchNormalization()(c2)
    d2 = Dropout(dropout_rate)(c2)
    p2 = MaxPool2D(pool_size = (2, 2))(d2) # Shape : (16, 16, 128)
    
    c3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_uniform')(p2) # Shape : (16, 16, 256)
    c3 = BatchNormalization()(c3)
    c3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_uniform')(c3) # Shape : (16, 16, 256)
    c3 = BatchNormalization()(c3)
    d3 = Dropout(dropout_rate)(c3)
    p3 = MaxPool2D(pool_size = (2, 2))(d3) # Shape : (8, 8, 256)

    # Bottleneck
    c4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_uniform')(p3) # Shape : (8, 8, 512)
    c4 = BatchNormalization()(c4)
    c4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_uniform')(c4) # Shape : (8, 8, 512)
    c4 = BatchNormalization()(c4)
    d4 = Dropout(dropout_rate)(c4)
    
    # Decoder (upsampling path)
    u5 = Conv2DTranspose(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_uniform')(d4) # Shape : (8, 8, 256)
    u5 = UpSampling2D((2, 2))(u5) # Shape : (16, 16, 256)
    concat5 = Concatenate()([u5, c3]) # Shape : (16, 16, 256)
    c5 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_uniform')(concat5) # Shape : (16, 16, 256)
    c5 = BatchNormalization()(c5)
    c5 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_uniform')(c5) # Shape : (16, 16, 256)
    c5 = BatchNormalization()(c5)
    d5 = Dropout(dropout_rate)(c5)
    
    u6 = Conv2DTranspose(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_uniform')(d5) # Shape : (16, 16, 128)
    u6 = UpSampling2D((2, 2))(u6) # Shape : (32, 32, 128)
    concat6 = Concatenate()([u6, c2]) # Shape : (32, 32, 128)
    c6 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_uniform')(concat6) # Shape : (32, 32, 128)
    c6 = BatchNormalization()(c6)
    c6 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_uniform')(c6) # Shape : (32, 32, 128)
    c6 = BatchNormalization()(c6)
    d6 = Dropout(dropout_rate)(c6)
    
    u7 = Conv2DTranspose(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_uniform')(d6) # Shape : (32, 32, 64)
    u7 = UpSampling2D((2, 2))(u7) # Shape : (64, 64, 64)
    concat7 = Concatenate()([u7, c1])  # Shape : (64, 64, 64)
    c7 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_uniform')(concat7) # Shape : (64, 64, 64)
    c7 = BatchNormalization()(c7)
    c7 = Conv2D(1, 3, activation = 'relu', padding = 'same', kernel_initializer = 'glorot_uniform')(c7) # Shape : (64, 64, 1)
    c7 = BatchNormalization()(c7)
    
    # Output layer
    flat = Flatten()(c7) # Shape : 64*64*1
    output = Dense(1, activation = 'linear')(flat) # Shape : 1
    
    model = Model(inputs = inputs, outputs = output)
    
    return model

# Creating and checking if saved_models folder exists or not. If not, then create a folder where we will store our weights.h5 file.
checkpoint_dir = './saved_models/' 
os.makedirs(checkpoint_dir, exist_ok = True)

best_model_filepath = os.path.join(checkpoint_dir, 'modelweights_lip.h5')
# Callbacks can be of best weights or last weights. We can do either one. We went with saving the weights after every epoch. 
best_model_checkpoint_callback = ModelCheckpoint(filepath = best_model_filepath, save_weights_only = True, verbose = 1)

# Applying data augmentation to Training data.
data_augmentation = ImageDataGenerator(rotation_range = 180, width_shift_range = 0.0, height_shift_range = 0.0, shear_range = 90, zoom_range = 0.2, horizontal_flip = True, vertical_flip = True, fill_mode = 'nearest')

# Filepaths for Train, Valid, Test. From the paper we know that for Train, Valid we will choose data points from everywhere except the location in which
# we will test our model's performance. Location = Lippendorf
test_set = 'curated/lip_test_dataset.nc'
valid_set = 'curated/lip_valid_dataset.nc'
train_set = 'curated/lip_train_dataset.nc'

# Here, we have generated our custom dataset.
train_dataset = CustomDataset(train_set, variable = 'no2_noisy')
valid_dataset = CustomDataset(valid_set, variable = 'no2_noisy')
test_dataset = CustomDataset(test_set, variable = 'no2_noisy')

# To store the inputs and outputs from our dataset to feed it to the model during training, validation and inference.
train_inputs = []
train_outputs = []
val_inputs = []
val_outputs = []
test_inputs = []
test_outputs = []

for inputs, outputs in train_dataset:
    train_inputs.append(inputs)
    train_outputs.append(outputs)
for inputs, outputs in valid_dataset:
    val_inputs.append(inputs)
    val_outputs.append(outputs)
for inputs, outputs in test_dataset:
    test_inputs.append(inputs)
    test_outputs.append(outputs)

# Converting into numpy arrays
train_inputs = np.array(train_inputs) # train_inputs shape : (25152, 64, 64, 4)
train_outputs = np.array(train_outputs) # train_outputs shape : (25152, 1)
val_inputs = np.array(val_inputs) # val_inputs shape : (4608, 64, 64, 4)
val_outputs = np.array(val_outputs) # val_outputs shape : (4608, 1)
test_inputs = np.array(test_inputs) # test_inputs shape : (6289, 64, 64, 4)
test_outputs = np.array(test_outputs) # test_outputs shape : (6289, 1)

model = unetreg(input_shape = (64, 64, 4), dropout_rate = 0.2) # Declaring the model
#model.load_weights('saved_models/modelweights_lip.h5') # If we have already stored the weights, then uncomment this line to load.

optimizer = Adam(learning_rate = 1e-3) # Taking Adam optimizer with lr = 1e-3

# We will update our lr if we reach a plateau. For this we use ReduceLROnPlateau which will monitor val_loss, wait for 20 epochs before changing lr and measures 
# min_delta threshold, reduces lr by 0.5 and lower bound on lr is 5e-5.
learning_rate_monitor_callback = ReduceLROnPlateau(monitor = 'val_loss', factor = 0.5, patience = 20, verbose = 1, min_delta = 5e-3, cooldown = 0, min_lr = 5e-5)

# Loss function for this task is MeanAbsoluteError and metrics are MeanAbsolutePercentageError, MeanSquaredError.
model.compile(optimizer = optimizer, loss = 'mae', metrics = ['mape'])

# Generate the summary of the model. Around 8 Million trainable paramaters are there.
model.summary()

# Total epochs = 200, batch size for training set = 32, steps will be 25152/32 = 786, validation will not have any augmentation with batch size of 32 and validating
# on the entire validation set.
history = model.fit(data_augmentation.flow(train_inputs, train_outputs, batch_size = 32, shuffle = True), epochs = 120, steps_per_epoch = len(train_inputs)//32,
validation_data = (val_inputs, val_outputs), validation_batch_size = 32, validation_steps = None, callbacks = [learning_rate_monitor_callback, best_model_checkpoint_callback])

# To test the model's performance on test set.
test_loss, test_metrics = model.evaluate(test_inputs, test_outputs, batch_size = None) # Uncomment these lines to infer model's performance.
print(f'Test Loss: {test_loss}, Test MAPE: {test_metrics}')

# To store the true_emission and predicted emissions for the .csv file.
true_emissions = []
predicted_emissions = []
for inputs, targets in test_dataset:
    inputs = np.expand_dims(inputs, axis = 0)
    outputs = model.predict(inputs) # Predicting the output based on the model's learning.
    print("True value : ", targets)
    print("Predicted value : ", outputs)
    true_emissions.append(targets) # Append the true emissions into true_emissions list
    predicted_emissions.append(outputs) # Append the predicted emissions into predicted_emissions list

# Converting into numpy arrays
true_emissions = np.array(true_emissions) # Shape : 6289, 3
predicted_emissions = np.array(predicted_emissions) # Shape : 6289, 1, 3
true_emissions = true_emissions.reshape(predicted_emissions.shape) # After reshaping : 6289, 1, 3

# Storing the true emissions and predicted emissions in a csv file. 
df = pd.DataFrame({'True Emissions': true_emissions.flatten(), 'Predicted Emissions': predicted_emissions.flatten()})

df.to_csv('emissions_results_lip.csv')
#print(df)


In [ ]:
# importing necessary package
import pandas as pd

""" After doing it twice, we now look at error, absolute error, relative errors. """ 
results_df = pd.read_csv('emissions_results_lip.csv') # Reading emissions_results

# Creating new column error which is true-pred.
results_df['error'] = (results_df['True Emissions'] - results_df['Average_Emissions']).round(3)

# Creating new column absolute_error which is the absolute value of error rounded off to 3 decimal places.
results_df['absolute_error'] = (abs(results_df['error'])).round(3)

# Creating new column relative_error which is the relative value of error rounded off to 3 decimal places with epsilon = 1e-15.
results_df['relative_error'] = ((results_df['error'] / (results_df['True Emissions'] + 1e-15)) * 100).round(3)

# Updates the csv file
results_df.to_csv('emissions_results_lip.csv', index = False)

In [ ]:
# importing necessary package
import pandas as pd

df = pd.read_csv('emissions_results_lip.csv') # Reading emissions_results

# We want to know the Q1, Q2, Q3 values or (25%, 50%, 75%) values from our absolute_error distribution.
percentiles = df['absolute_error'].describe()
#percentiles = [0.25, 0.5, 0.75]
percentiles['25%'] = abs(percentiles['25%'])
percentiles['50%'] = abs(percentiles['50%'])
percentiles['75%'] = abs(percentiles['75%'])

# Only the percentiles are of impportant to us.
#percentiles = percentiles.loc[['25%', '50%', '75%']].apply(abs) # 25% : 2.53, 50% : 5.07, 75% : 7.94 (All are in Mt/yr)

# Stated in the paper, the median absolute error is 3 Mt/yr. We got 2.3 Mt/ yr which means we are at a -0.74 Mt/yr deviation which is better than the paper.

# The true percentile values as mentioned in the paper.
true_percentiles = [1.3, 2.7, 4.5] # (All are in Mt/yr)

# Absolute error between our percentile values and the true percentile values rounded off to 3 decimal places.
error = [round(abs(true - pred), 3) for true, pred in zip(true_percentiles, percentiles)] # 1.23, 2.37, 3.4

print(percentiles)
print("Error:", error)

In [ ]:
# importing necessary package
import pandas as pd

df = pd.read_csv('emissions_results_lip.csv') # Reading emissions_results
df['abs_rel_error'] = abs(df['relative_error'])

# We want to know the Q1, Q2, Q3 values or (25%, 50%, 75%) values from our relative_error distribution.
percentiles = df['abs_rel_error'].describe()
#percentiles = [0.25, 0.5, 0.75]
# Only the percentiles are of impportant to us.
#percentiles = percentiles.loc[['25%', '50%', '75%']].apply(abs) # 25% : 17.1, 50% : 34.8, 75% : 52.0 (Unitless)

# Stated in the paper, the median absolute relative paper is around 20%. We have got 14.76% which can be approximated to 15%.
# This means our model performed better.

# The true percentile values as mentioned in the paper.
true_percentiles = [8.6, 18.1, 30.3] # Note : These values are percentage values as the relative errors are calculated in terms of percentage.

# Absolute error between our percentile values and the true percentile values rounded off to 3 decimal places.
error = [round(abs(true - pred), 3) for true, pred in zip(true_percentiles, percentiles)] # 3.6, 6.35, 10.38 
#df.to_csv('emissions_results.csv', index = False)
print(percentiles)
print("Error:", error)

In [ ]:
""" This cell is for plotting the Kernel Density Estimation (KDE) plots using seaborn of absolute_error and relative_error. """

# importing necessary packages
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

df1 = pd.read_csv('emissions_results_lip.csv') # Reading emissions_results
df2 = pd.read_csv('emissions_data_lip.csv') # Reading emissions_data

plt.figure(figsize = (10, 6))
sns.kdeplot(df1['absolute_error'], fill = True, color = 'b', label = 'Using U-Net regression') # Using seaborn's kdeplot.
sns.kdeplot(df2['absolute_error'], fill = True, color = 'r', label = 'Using CNN regression')
plt.title('Kernel Density Estimate of Absolute Error for Lippendorf')
plt.xlabel('Absolute Error')
plt.ylabel('Density')
plt.legend()
plt.xlim(0,25)
plt.savefig('absolute_error.png', dpi = 300, bbox_inches = 'tight') # Saving the file
plt.show()

plt.figure(figsize = (10, 6))
sns.kdeplot(df1['relative_error'], fill = True, color = 'b', label = 'Using U-Net regression') # Using seaborn's kdeplot.
sns.kdeplot(df2['relative_error'], fill = True, color = 'r', label = 'Using CNN regression')
plt.title('Kernel Density Estimate of Relative Error (%) for Lippendorf')
plt.xlabel('Relative Error (%)')
plt.ylabel('Density')
plt.legend()
plt.xlim(-110,120)
plt.savefig('relative_error.png', dpi = 300, bbox_inches = 'tight') # Saving the file
plt.show()

In [ ]:
""" This code is to know how many of our absolute errors lie in (0-2) Mt/yr range, in (2-5) Mt/yr range, in (5-10) Mt/yr range
and in 10 Mt/yr above. Along with that, let us also get some statistics. """

# importing necessary package
import pandas as pd

# Read csv file
df = pd.read_csv('emissions_results_lip.csv')

# useful column
abs = df['absolute_error']

# get the required info
in0_2 = (abs >= 0) & (abs <= 2)
in2_5 = (abs > 2) & (abs <= 5)
in5_10 = (abs > 5) & (abs <= 10)
ab10 = abs > 10
mean = round(abs.mean(), 3)
median = round(abs.median(), 3)
std = round(abs.std(), 3)

print("in between 0 and 2 : ", in0_2.sum()) # 1222
print("in between 2 and 5 : ", in2_5.sum()) # 1888
print("in between 5 and 10 : ", in5_10.sum()) # 2425
print("above 10 : ", ab10.sum()) # 754
print("mean : ", mean) # 5.55
print("median : ", median) # 5.071
print("std : ", std) # 3.77

# According to the paper, " Majority of errors are concentrated below 10 Mt/yr." which we can see is true. In our case, we can say that majority of errors are
# below 5 Mt/yr which is a significant increase in performance from 10 Mt/yr.

In [ ]:
""" This code is to know how many of our relative errors lie less than -150%, in (-150 to -100)% range, in (-100 to -50)% range,
in (-50 to 0) %, in (0 to 50) % range, in (50 to 100) % and above 100%. Along with that, let us also get some statistics. """

# importing necessary package
import pandas as pd

# Read csv file
df = pd.read_csv('emissions_results_lip.csv')

# useful column
rel = df['relative_error']

# get the required info
lessneg150 = (rel <= -150)
neg150_100 = (rel > -150) & (rel <= -100)
neg100_50 = (rel > -100) & (rel <= -50)
neg50_0 = (rel > -50) & (rel <= 0)
in0_50 = (rel > 0) & (rel <= 50)
in50_100 = (rel > 50) & (rel <= 100)
ab100 = rel > 100
mean = round(rel.mean(), 3)
median = round(rel.median(), 3)
std = round(rel.std(), 3)

print("less than -150% : ", lessneg150.sum()) # 33
print("in between -150% and -100% : ", neg150_100.sum()) # 164
print("in between -100% and -50% : ", neg100_50.sum()) # 778
print("in between -50% and 0% : ", neg50_0.sum()) # 2627
print("in between 0% and 50% : ", in0_50.sum()) # 2627
print("in between 50% and 100% : ", in50_100.sum()) # 776
print("above 100% : ", ab100.sum()) # 0
print("mean : ", mean) # -1.56
print("median : ", median) # 5.38
print("std : ", std) # 46.5

In [ ]:
""" This code is to know how many of our absolute relative errors lie in (0 to 20) % range, in (20 to 50)% range, in (50 to 100) %, in (100 to 150)% 
and above 150%. Along with that, let us also get some statistics. """

# importing necessary package
import pandas as pd

# Read csv file
df = pd.read_csv('emissions_results_lip.csv')
df['abs_rel_error'] = abs(df['relative_error'])
# useful column
rel = df['abs_rel_error']

# get the required info
above150 = (rel > 150)
in150_100 = (rel > 100) & (rel <= 150)
in100_50 = (rel > 50) & (rel <= 100)
in50_20 = (rel > 20) & (rel <= 50)
in20_0 = (rel > 0) & (rel <= 20)

mean = round(rel.mean(), 3)
median = round(rel.median(), 3)
std = round(rel.std(), 3)

print("above 150% : ", above150.sum()) # 33
print("in between 100% and 150% : ", in150_100.sum()) # 163
print("in between 50% and 100% : ", in100_50.sum()) # 1545
print("in between 20% and 50% : ", in50_20.sum()) # 2734
print("in between 0% and 20% : ", in20_0.sum()) # 1814
print("mean : ", mean) # 37.7
print("median : ", median) # 34.8
print("std : ", std) # 27.27

# According to the paper, " Majority of errors are concentrated below 10 Mt/yr or 50% with very few exceeding 100%." which we can again see is true
# In our model also, Majority of the errors are below 50%.

In [ ]:
""" This code gets us a few statistics about true emissions and predicted emissions. """

# necessary package
import pandas as pd

# read
df = pd.read_csv('emissions_results_lip.csv')

true = df['True Emissions']
min_true = round(true.min(), 3)
max_true = round(true.max(), 3)
mean_true = round(true.mean(), 3)
std_true = round(true.std(), 3)
median_true = round(true.median(), 3)
range_true = round(max_true-min_true, 3)

pred = df['Average_Emissions']
min_pred = round(pred.min(), 3)
max_pred = round(pred.max(), 3)
mean_pred = round(pred.mean(), 3)
std_pred = round(pred.std(), 3)
median_pred = round(pred.median(), 3)
range_pred = round(max_pred-min_pred, 3)

print("True statistics : ")
print()
print("min : ", min_true) # 7.493
print("max : ", max_true) # 24.112
print("mean : ", mean_true) # 15.256
print("std : ", std_true) # 3.527
print("median : ", median_true) # 15.264
print("range : ", range_true) # 16.619
print()
print("Predicted statistics : ")
print()
print("min : ", min_pred) # 3.59
print("max : ", max_pred) # 41.75
print("mean : ", mean_pred) # 15.23
print("std : ", std_pred) # 7.1
print("median : ", median_pred) # 14.23
print("range : ", range_pred) # 38.16

# The mean true emission = 15.2 Mt/yr and mean pred emission = 14.3 Mt/yr which tantamounts to the fact that our model has come closer to the real true emission values.

Redistributing the dataset to get our own datasets

Working with CNN regression model

In [ ]:
"""
Importing different packages that we will use.

"""
import h5py
import os
import numpy as np
from netCDF4 import Dataset as NetCDFDataset
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, LeakyReLU, Dropout, BatchNormalization, MaxPool2D, Flatten, Dense, GaussianNoise
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tqdm import tqdm
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
import pandas as pd
import tensorflow as tf

"""
We define our Custom Dataset which will read the .nc files (train, valid, test) and extract the XCO2, u, v, NO2 data from the .nc files.
Note to the reader : For our comparision, we are going to consider Lippendorf location and as additional input we will consider NO2 images.

"""
class CustomDataset:
    
    def __init__(self, file_path, variable = None):
        self.ncfile = NetCDFDataset(file_path, 'r') # Reading the .nc file
        self.variable = variable # Variable in our comparison is no2_noisy
    
    def __len__(self):
        return self.ncfile.variables['xco2_noisy'].shape[0] # The total number of images or data we will have which is different for train (25152), valid (4608), test (6289)
    
    def __getitem__(self, idx):

        # Reading the necessary variables (xco2_noisy, u, v)
        xco2_read = self.ncfile.variables['xco2_noisy'][idx] # Unit : ppmv (parts per million by volume), Shape : 64, 64
        u_read = self.ncfile.variables['u'][idx] # Unit : m/s, Shape : 64, 64
        v_read = self.ncfile.variables['v'][idx] # Unit : m/s, Shape : 64, 64
        emiss_read = self.ncfile.variables['emiss'][idx] # emiss is the true output data that we need to train our model. Unit : Mt/yr (Million tonnes/ year)
                                                         # Shape : 3,
        # Converting into numpy arrays with data type float32
        xco2_arr = np.array(xco2_read.data).astype('float32') 
        u_arr = np.array(u_read.data).astype('float32')
        v_arr = np.array(v_read.data).astype('float32')
        emiss_arr = np.array(emiss_read.data).astype('float32')

        #noise = GaussianNoise(stddev = 0.7) # Addition of Gaussian Noise with std of 0.7
        #xco2 = noise(xco2_arr).numpy() 
        if self.variable:
            var_read = self.ncfile.variables[self.variable][idx] # Unit : molec/cm^2, Shape : 64, 64
            var_arr = np.array(var_read.data).astype('float32') # Converting into numpy array after reading the variable
            #var_noise = noise(var_tensor).numpy()
            inputs = np.stack([xco2_arr, u_arr, v_arr, var_arr], axis = -1) # Stacking the inputs to get the desired input shape = (64, 64, 4) where 64, 64
                                                                            # represents height, width and 4 represents the total number of features. Each pixel
                                                                            # has a spatial resolution of 2km, i.e. 1*1 in pixel refers to an area of 2km*2km.
        else: 
            inputs = np.stack([xco2_arr, u_arr, v_arr], axis = -1) # Stacking the inputs to get the shape = (64, 64, 3) in case when variable is None.
        
        mean = inputs.mean(axis = (0, 1), keepdims = True) # Calculating the mean of the 4 features separately and keeping the same shape as that of inputs.
        
        std = inputs.std(axis = (0, 1), keepdims = True) # Calculating the std of the 4 features separately and keeping the same shape as that of inputs.
        
        std[std==0] = 1 # If std becomes 0, we change it 1 to avoid division by 0. Multiple other methods can also be employed such as choosing an 
                        # appropriate epsilon such as 1e-5, 1e-6 or 1e-7
        
        inputs = (inputs - mean)/std # Normalization of the inputs separately for each feature.
        
        # emiss data is of shape (3,), from which we will only consider the middle value for our comparison. Hence, we multiply weights with emiss to
        # get the middle value and round it off to 3 decimal places.
        weights = np.array([0.0, 1.0, 0.0]) 
        weighted = np.round(np.sum(weights * emiss_arr), 3)
        outputs = np.array([weighted], dtype = np.float32)
        
        return inputs, outputs # returns our inputs (64, 64, 4), outputs (1,)

    def __del__(self):
        self.ncfile.close() # Closing the ncfile to avoid corruption of the file.

"""
The model architecture is taking from the paper which consists of 2D convolution, Maxpooling, Dropout, Batch Normalization, Flatten and finally Dense.

"""
def model_arch(input_shape): # input_shape = 64, 64, 4
    model = Sequential()
    
    # The first convolution with kernel size of (3,3), total number of filters = 32, activation function = elu (Exponential Linear Unit), strides = 1, 
    # padding = valid (no padding).
    model.add(Conv2D(32, (3, 3), activation = "elu", strides = 1, input_shape = input_shape)) # Shape after conv : 62, 62, 32
    
    # Adding a Dropout of 0.1
    model.add(Dropout(0.1)) # No change in shape
    
    # Second convolution with kernel size of (3,3), total number of filters = 32, activation function = elu (Exponential Linear Unit), strides = 1,
    # padding = valid (no padding)
    model.add(Conv2D(32, (3, 3), activation = "elu", strides = 1)) # Shape after conv : 60, 60, 32
    
    # Maxpool with pool size of 2, 2 and strides = 2.
    model.add(MaxPool2D(pool_size = (2, 2), padding = "valid", strides = 2)) # Shape after maxpool : 30, 30, 32
    
    model.add(BatchNormalization()) # Batch Normalization which does not affect the shape
    
    # Third convolution with kernel size of (3,3), total number of filters = 32, activation function = elu (Exponential Linear Unit), strides = 1,
    # padding = valid (no padding)
    model.add(Conv2D(32, (3, 3), activation = "elu", strides = 1)) # Shape after conv : 28, 28, 32
    
    # Adding a Dropout of 0.2
    model.add(Dropout(0.2))
    
    # Fourth convolution with kernel size of (3,3), total number of filters = 64, activation function = elu (Exponential Linear Unit), strides = 1,
    # padding = valid (no padding)
    model.add(Conv2D(64, (3, 3), activation = "elu", strides = 1)) # Shape after conv : 26, 26, 64
    
    model.add(BatchNormalization()) # Batch Normalization which does not affect the shape
    
    # Fifth convolution with kernel size of (3,3), total number of filters = 64, activation function = elu (Exponential Linear Unit), strides = 1,
    # padding = valid (no padding)
    model.add(Conv2D(64, (3, 3), activation = "elu", strides = 1)) # Shape after conv : 24, 24, 64
    
    # Adding a Dropout of 0.2
    model.add(Dropout(0.2)) # No change in shape
    
    # Sixth convolution with kernel size of (3,3), total number of filters = 64, activation function = elu (Exponential Linear Unit), strides = 1,
    # padding = valid (no padding)
    model.add(Conv2D(64, (3, 3), activation = "elu", strides = 1)) # Shape after conv : 22, 22, 64
    
    # Maxpool with pool size of 2, 2 and strides = 2.
    model.add(MaxPool2D(pool_size = (2, 2), padding = "valid", strides = 2)) # Shape after maxpool : 11, 11, 64
    
    # Seventh convolution with kernel size of (3,3), total number of filters = 64, activation function = elu (Exponential Linear Unit), strides = 1,
    # padding = valid (no padding)
    model.add(Conv2D(64, (3, 3), activation = "elu", strides = 1)) # Shape after conv : 9, 9, 64
    
    # Adding a Dropout of 0.2
    model.add(Dropout(0.2))
    
    # Eighth convolution with kernel size of (3,3), total number of filters = 64, activation function = elu (Exponential Linear Unit), strides = 1,
    # padding = valid (no padding)
    model.add(Conv2D(64, (3, 3), activation = "elu", strides = 1)) # Shape after conv : 7, 7, 64
    
    # Maxpool with pool size of 2, 2 and strides = 2.
    model.add(MaxPool2D(pool_size = (2, 2), padding = "valid", strides = 2)) # Shape after maxpool : 3, 3, 64
    
    model.add(Flatten()) # Total = 3*3*64 = 576
    
    model.add(Dense(1)) # Fully Connected Layer to get a single output.
    
    model.add(LeakyReLU(alpha = 0.3)) # The output goes through activation function = Leaky Rectified Linear Unit with negative slope = 0.3.
    
    return model

# Creating and checking if saved_models folder exists or not. If not, then create a folder where we will store our weights.h5 file.
checkpoint_dir = './saved_models/' 
os.makedirs(checkpoint_dir, exist_ok = True)

best_model_filepath = os.path.join(checkpoint_dir, 'best_model_weights_new.h5')
# Callbacks can be of best weights or last weights. We can do either one. We went with saving the weights after every epoch. 
best_model_checkpoint_callback = ModelCheckpoint(filepath = best_model_filepath, save_weights_only = True, verbose = 1)

# Applying data augmentation to Training data as specified by the paper.
data_augmentation = ImageDataGenerator(rotation_range = 180, width_shift_range = 0.0, height_shift_range = 0.0, shear_range = 90, zoom_range = 0.2, horizontal_flip = True, vertical_flip = True, fill_mode = 'nearest')

# Filepaths for Train, Valid, Test. From the paper we know that for Train, Valid we will choose data points from everywhere except the location in which
# we will test our model's performance. Location = Lippendorf
test_set = 'curated/lip_test_dataset.nc'
valid_set = 'curated/lip_valid_dataset.nc'
train_set = 'curated/lip_train_dataset.nc'

# Here, we have generated our custom dataset.
train_dataset = CustomDataset(train_set, variable = 'no2_noisy')
valid_dataset = CustomDataset(valid_set, variable = 'no2_noisy')
test_dataset = CustomDataset(test_set, variable = 'no2_noisy')

# To store the inputs and outputs from our dataset.
combine_inputs = []
combine_outputs = []

for dataset in [train_dataset, valid_dataset, test_dataset]:
    for inputs, outputs in dataset:
        combine_inputs.append(inputs)
        combine_outputs.append(outputs)

# Converting into numpy arrays
combine_inputs = np.array(combine_inputs) # Shape : (36049, 64, 64, 4)
combine_outputs = np.array(combine_outputs) # Shape : (36049, 1)

indices = np.arange(combine_inputs.shape[0])
np.random.shuffle(indices)
combine_inputs = combine_inputs[indices]
combine_outputs = combine_outputs[indices]

# Train, Val, Test lists
train_size = int(0.65 * combine_inputs.shape[0])
valid_size = int(0.15 * combine_inputs.shape[0])
test_size = combine_inputs.shape[0] - train_size - valid_size

train_inputs, val_inputs, test_inputs = np.split(combine_inputs, [train_size, train_size + valid_size])
train_outputs, val_outputs, test_outputs = np.split(combine_outputs, [train_size, train_size + valid_size])

# Delete unnecessary variables to free up memory
del combine_inputs
del combine_outputs
del train_dataset
del valid_dataset
del test_dataset
del indices
del train_size
del valid_size
del test_size

# Shapes for our data : 
# Train input Shape : (23431, 64, 64, 4), Train output Shape : (23431, 1)
# Valid input Shape : (5407, 64, 64, 4), Valid output Shape :  (5407, 1)
# Test input Shape : (7211, 64, 64, 4), Test output Shape : (7211, 1)

model = model_arch(input_shape = (64, 64, 4)) # Declaring the model

model.load_weights('saved_models/best_model_weights_new.h5') # If we have already stored the weights, then uncomment this line to load.

optimizer = Adam(learning_rate = 1e-3) # Taking Adam optimizer with lr = 1e-3

# We will update our lr if we reach a plateau. For this we use ReduceLROnPlateau which will monitor val_loss, wait for 20 epochs before changing lr and measures 
# min_delta threshold, reduces lr by 0.5 and lower bound on lr is 5e-5.
learning_rate_monitor_callback = ReduceLROnPlateau(monitor = 'val_loss', factor = 0.5, patience = 20, verbose = 1, min_delta = 5e-3, cooldown = 0, min_lr = 5e-5)

# Loss function for this task is MeanAbsoluteError and metrics is MeanAbsolutePercentageError.
model.compile(optimizer = optimizer, loss = 'mae', metrics = ['mape'])

# Generate the summary of the model. Around 186000 trainable paramaters are there.
model.summary()

# Total epochs = 500, batch size for training set = 32, steps will be 23431/32 = 732, validation will not have any augmentation with batch size of 32 and validating
# on the entire validation set.
history = model.fit(data_augmentation.flow(train_inputs, train_outputs, batch_size = 32, shuffle = True), epochs = 500, steps_per_epoch = len(train_inputs)//32,
validation_data = (val_inputs, val_outputs), validation_batch_size = 32, validation_steps = None, callbacks = [learning_rate_monitor_callback, best_model_checkpoint_callback])

# To test the model's performance on test set.
test_loss, test_mae = model.evaluate(test_inputs, test_outputs, batch_size = None) # Uncomment these lines to infer model's performance.
print(f'Test Loss: {test_loss}, Test MAE: {test_mae}')

# To store the true_emission and predicted emissions for the .csv file.
true_emissions = []
predicted_emissions = []
for i in range(len(test_inputs)):
    inputs = np.expand_dims(test_inputs[i], axis = 0)
    outputs = model.predict(inputs)  # Predicting the output based on the model's learning.
    print("True value : ", test_outputs[i])
    print("Predicted value : ", outputs)
    true_emissions.append(test_outputs[i])  # Append the true emissions into true_emissions list
    predicted_emissions.append(outputs)  # Append the predicted emissions into predicted_emissions list

# Converting into numpy arrays
true_emissions = np.array(true_emissions)  # Shape : 7211, 3
predicted_emissions = np.array(predicted_emissions)  # Shape : 7211, 1, 3
true_emissions = true_emissions.reshape(predicted_emissions.shape)  # After reshaping : 7211, 1, 3

# Storing the true emissions and predicted emissions in a csv file.
df = pd.DataFrame({'True Emissions': true_emissions.flatten(), 'Predicted Emissions': predicted_emissions.flatten()}, index = np.arange(1, len(true_emissions) + 1))
df.to_csv('emiss_data.csv')
print(df)


In [ ]:
import numpy as np

train_outputs = np.load('train_comb_outputs.npy')
valid_outputs = np.load('valid_comb_outputs.npy')
test_outputs = np.load('test_comb_outputs.npy')

def compute_statistics(data):
    
    
    total_min = np.min(data)
    total_max = np.max(data) 
    total_range = total_max - total_min
    total_mean = np.mean(data)
    total_median = np.median(data)
    
    return {
        
        'total_min': total_min,
        'total_max': total_max,
        'total_range': total_range,
        'total_mean': total_mean,
        'total_median': total_median
    }

train_stats = compute_statistics(train_outputs)
valid_stats = compute_statistics(valid_outputs)
test_stats = compute_statistics(test_outputs)

print("Train Outputs Statistics:")
for key, value in train_stats.items():
    print(f"{key}: {value}")

print("\nValidation Outputs Statistics:")
for key, value in valid_stats.items():
    print(f"{key}: {value}")

print("\nTest Outputs Statistics:")
for key, value in test_stats.items():
    print(f"{key}: {value}")

bins = np.concatenate([np.arange(0, 5.5, 0.5), np.arange(5, 25, 2), [np.inf]])

def calculate_percentage(data, bins):
    counts, _ = np.histogram(data, bins = bins)
    total_count = np.sum(counts)
    print(total_count)
    percentages = (counts / total_count) * 100
    return counts, percentages

train_counts, train_percentages = calculate_percentage(train_outputs.flatten(), bins)
valid_counts, valid_percentages = calculate_percentage(valid_outputs.flatten(), bins)
test_counts, test_percentages = calculate_percentage(test_outputs.flatten(), bins)

def stats(name, counts, percentages, bins):
    print(f'\n{name} Data Histogram:')
    for i in range(len(counts)):
        print(f'Bin range {bins[i]:.1f} to {bins[i+1]:.1f} (or {bins[i]} to {bins[i+1]}):')
        print(f'  Count: {counts[i]}')
        print(f'  Percentage: {percentages[i]:.2f}%')

stats('Train', train_counts, train_percentages, bins)
stats('Valid', valid_counts, valid_percentages, bins)
stats('Test', test_counts, test_percentages, bins)

Working with U-Net regression model

In [ ]:
""" This marks the beginning of our novel approach that is using U-Net for regression task. """
"""
Importing different packages that we will use.

"""
import h5py
import os
import numpy as np
from netCDF4 import Dataset as NetCDFDataset
from tensorflow.keras.layers import Input, Conv2D, Dropout, BatchNormalization, MaxPool2D, Concatenate, Conv2DTranspose, Flatten, Dense, UpSampling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tqdm import tqdm
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
import pandas as pd
import tensorflow as tf

"""
We define our Custom Dataset which will read the .nc files (train, valid, test) and extract the XCO2, u, v, NO2 data from the .nc files.
Note to the reader : For our comparision, we are going to consider Lippendorf location and as additional input we will consider NO2 images.

"""
class CustomDataset:
    
    def __init__(self, file_path, variable = None):
        self.ncfile = NetCDFDataset(file_path, 'r') # Reading the .nc file
        self.variable = variable # Variable in our comparison is no2_noisy
    
    def __len__(self):
        return self.ncfile.variables['xco2_noisy'].shape[0] # The total number of images or data we will have which is different for train (25152), valid (4608), test (6289)
    
    def __getitem__(self, idx):

        # Reading the necessary variables (xco2_noisy, u, v)
        xco2_read = self.ncfile.variables['xco2_noisy'][idx] # Unit : ppmv (parts per million by volume), Shape : 64, 64
        u_read = self.ncfile.variables['u'][idx] # Unit : m/s, Shape : 64, 64
        v_read = self.ncfile.variables['v'][idx] # Unit : m/s, Shape : 64, 64
        emiss_read = self.ncfile.variables['emiss'][idx] # emiss is the true output data that we need to train our model. Unit : Mt/yr (Million tonnes/ year)
                                                         # Shape : 3,
        # Converting into numpy arrays with data type float32
        xco2_arr = np.array(xco2_read.data).astype('float32') 
        u_arr = np.array(u_read.data).astype('float32')
        v_arr = np.array(v_read.data).astype('float32')
        emiss_arr = np.array(emiss_read.data).astype('float32')

        #noise = GaussianNoise(stddev = 0.7) # Addition of Gaussian Noise with std of 0.7
        #xco2 = noise(xco2_arr).numpy() 
        if self.variable:
            var_read = self.ncfile.variables[self.variable][idx] # Unit : molec/cm^2, Shape : 64, 64
            var_arr = np.array(var_read.data).astype('float32') # Converting into numpy array after reading the variable
            #var_noise = noise(var_tensor).numpy()
            inputs = np.stack([xco2_arr, u_arr, v_arr, var_arr], axis = -1) # Stacking the inputs to get the desired input shape = (64, 64, 4) where 64, 64
                                                                            # represents height, width and 4 represents the total number of features. Each pixel
                                                                            # has a spatial resolution of 2km, i.e. 1*1 in pixel refers to an area of 2km*2km.
        else: 
            inputs = np.stack([xco2_arr, u_arr, v_arr], axis = -1) # Stacking the inputs to get the shape = (64, 64, 3) in case when variable is None.
        
        mean = inputs.mean(axis = (0, 1), keepdims = True) # Calculating the mean of the 4 features separately and keeping the same shape as that of inputs.
        
        std = inputs.std(axis = (0, 1), keepdims = True) # Calculating the std of the 4 features separately and keeping the same shape as that of inputs.
        
        std[std==0] = 1 # If std becomes 0, we change it 1 to avoid division by 0. Multiple other methods can also be employed such as choosing an 
                        # appropriate epsilon such as 1e-5, 1e-6 or 1e-7
        
        inputs = (inputs - mean)/std # Normalization of the inputs separately for each feature.
        
        # emiss data is of shape (3,), from which we will only consider the middle value for our comparison. Hence, we multiply weights with emiss to
        # get the middle value and round it off to 3 decimal places.
        weights = np.array([0.0, 1.0, 0.0]) 
        weighted = np.round(np.sum(weights * emiss_arr), 3)
        outputs = np.array([weighted], dtype = np.float32)
        
        return inputs, outputs # returns our inputs (64, 64, 4), outputs (1,)

    def __del__(self):
        self.ncfile.close() # Closing the ncfile to avoid corruption of the file.

""" The U-Net regression Model """

def unetreg(input_shape, dropout_rate = 0.2):
    inputs = Input(input_shape) # If we use no2 then shape : (64, 64, 4), if we don't use no2 then shape : (64, 64, 3)
    
    # Encoder (downsampling path)
    c1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_uniform')(inputs) # Shape : (64, 64, 64)
    c1 = BatchNormalization()(c1)
    c1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_uniform')(c1) # Shape : (64, 64, 64)
    c1 = BatchNormalization()(c1)
    d1 = Dropout(dropout_rate)(c1) 
    p1 = MaxPool2D(pool_size = (2, 2))(d1) # Shape : (32, 32, 64) 
    
    c2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_uniform')(p1) # Shape : (32, 32, 128)
    c2 = BatchNormalization()(c2)
    c2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_uniform')(c2) # Shape : (32, 32, 128)
    c2 = BatchNormalization()(c2)
    d2 = Dropout(dropout_rate)(c2)
    p2 = MaxPool2D(pool_size = (2, 2))(d2) # Shape : (16, 16, 128)
    
    c3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_uniform')(p2) # Shape : (16, 16, 256)
    c3 = BatchNormalization()(c3)
    c3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_uniform')(c3) # Shape : (16, 16, 256)
    c3 = BatchNormalization()(c3)
    d3 = Dropout(dropout_rate)(c3)
    p3 = MaxPool2D(pool_size = (2, 2))(d3) # Shape : (8, 8, 256)

    # Bottleneck
    c4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_uniform')(p3) # Shape : (8, 8, 512)
    c4 = BatchNormalization()(c4)
    c4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_uniform')(c4) # Shape : (8, 8, 512)
    c4 = BatchNormalization()(c4)
    d4 = Dropout(dropout_rate)(c4)
    
    # Decoder (upsampling path)
    u5 = Conv2DTranspose(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_uniform')(d4) # Shape : (8, 8, 256)
    u5 = UpSampling2D((2, 2))(u5) # Shape : (16, 16, 256)
    concat5 = Concatenate()([u5, c3]) # Shape : (16, 16, 256)
    c5 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_uniform')(concat5) # Shape : (16, 16, 256)
    c5 = BatchNormalization()(c5)
    c5 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_uniform')(c5) # Shape : (16, 16, 256)
    c5 = BatchNormalization()(c5)
    d5 = Dropout(dropout_rate)(c5)
    
    u6 = Conv2DTranspose(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_uniform')(d5) # Shape : (16, 16, 128)
    u6 = UpSampling2D((2, 2))(u6) # Shape : (32, 32, 128)
    concat6 = Concatenate()([u6, c2]) # Shape : (32, 32, 128)
    c6 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_uniform')(concat6) # Shape : (32, 32, 128)
    c6 = BatchNormalization()(c6)
    c6 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_uniform')(c6) # Shape : (32, 32, 128)
    c6 = BatchNormalization()(c6)
    d6 = Dropout(dropout_rate)(c6)
    
    u7 = Conv2DTranspose(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_uniform')(d6) # Shape : (32, 32, 64)
    u7 = UpSampling2D((2, 2))(u7) # Shape : (64, 64, 64)
    concat7 = Concatenate()([u7, c1])  # Shape : (64, 64, 64)
    c7 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_uniform')(concat7) # Shape : (64, 64, 64)
    c7 = BatchNormalization()(c7)
    c7 = Conv2D(1, 3, activation = 'relu', padding = 'same', kernel_initializer = 'glorot_uniform')(c7) # Shape : (64, 64, 1)
    c7 = BatchNormalization()(c7)
    
    # Output layer
    flat = Flatten()(c7) # Shape : 64*64*1
    output = Dense(1, activation = 'linear')(flat) # Shape : 1
    
    model = Model(inputs = inputs, outputs = output)
    
    return model

# Creating and checking if saved_models folder exists or not. If not, then create a folder where we will store our weights.h5 file.
checkpoint_dir = './saved_models/' 
os.makedirs(checkpoint_dir, exist_ok = True)

best_model_filepath = os.path.join(checkpoint_dir, 'modelweights_new.h5')
# Callbacks can be of best weights or last weights. We can do either one. We went with saving the weights after every epoch. 
best_model_checkpoint_callback = ModelCheckpoint(filepath = best_model_filepath, save_weights_only = True, verbose = 1)

# Applying data augmentation to Training data.
data_augmentation = ImageDataGenerator(rotation_range = 180, width_shift_range = 0.0, height_shift_range = 0.0, shear_range = 90, zoom_range = 0.2, horizontal_flip = True, vertical_flip = True, fill_mode = 'nearest')

# Filepaths for Train, Valid, Test. From the paper we know that for Train, Valid we will choose data points from everywhere except the location in which
# we will test our model's performance. Location = Lippendorf
test_set = 'curated/lip_test_dataset.nc'
valid_set = 'curated/lip_valid_dataset.nc'
train_set = 'curated/lip_train_dataset.nc'

# Here, we have generated our custom dataset.
train_dataset = CustomDataset(train_set, variable = 'no2_noisy')
valid_dataset = CustomDataset(valid_set, variable = 'no2_noisy')
test_dataset = CustomDataset(test_set, variable = 'no2_noisy')

# To store the inputs and outputs from our dataset.
combine_inputs = []
combine_outputs = []

for dataset in [train_dataset, valid_dataset, test_dataset]:
    for inputs, outputs in dataset:
        combine_inputs.append(inputs)
        combine_outputs.append(outputs)

# Converting into numpy arrays
combine_inputs = np.array(combine_inputs) # Shape : (36049, 64, 64, 4)
combine_outputs = np.array(combine_outputs) # Shape : (36049, 1)

indices = np.arange(combine_inputs.shape[0])
np.random.shuffle(indices)
combine_inputs = combine_inputs[indices]
combine_outputs = combine_outputs[indices]

# Train, Val, Test lists
train_size = int(0.65 * combine_inputs.shape[0])
valid_size = int(0.15 * combine_inputs.shape[0])
test_size = combine_inputs.shape[0] - train_size - valid_size

train_inputs, val_inputs, test_inputs = np.split(combine_inputs, [train_size, train_size + valid_size])
train_outputs, val_outputs, test_outputs = np.split(combine_outputs, [train_size, train_size + valid_size])

# Delete unnecessary variables to free up memory
del combine_inputs
del combine_outputs
del train_dataset
del valid_dataset
del test_dataset
del indices
del train_size
del valid_size
del test_size

# Shapes for our data : 
# Train input Shape : (23431, 64, 64, 4), Train output Shape : (23431, 1)
# Valid input Shape : (5407, 64, 64, 4), Valid output Shape :  (5407, 1)
# Test input Shape : (7211, 64, 64, 4), Test output Shape : (7211, 1)

model = unetreg(input_shape = (64, 64, 4), dropout_rate = 0.2) # Declaring the model
#model.load_weights('saved_models/modelweights_new.h5') # If we have already stored the weights, then uncomment this line to load.

optimizer = Adam(learning_rate = 1e-3) # Taking Adam optimizer with lr = 1e-3

# We will update our lr if we reach a plateau. For this we use ReduceLROnPlateau which will monitor val_loss, wait for 20 epochs before changing lr and measures 
# min_delta threshold, reduces lr by 0.5 and lower bound on lr is 5e-5.
learning_rate_monitor_callback = ReduceLROnPlateau(monitor = 'val_loss', factor = 0.5, patience = 20, verbose = 1, min_delta = 5e-3, cooldown = 0, min_lr = 5e-5)

# Loss function for this task is MeanAbsoluteError and metrics are MeanAbsolutePercentageError, MeanSquaredError.
model.compile(optimizer = optimizer, loss = 'mae', metrics = ['mape'])

# Generate the summary of the model. Around 8 Million trainable paramaters are there.
model.summary()

# Total epochs = 200, batch size for training set = 32, steps will be 25152/32 = 786, validation will not have any augmentation with batch size of 32 and validating
# on the entire validation set.
history = model.fit(data_augmentation.flow(train_inputs, train_outputs, batch_size = 32, shuffle = True), epochs = 200, steps_per_epoch = len(train_inputs)//32,
validation_data = (val_inputs, val_outputs), validation_batch_size = 32, validation_steps = None, callbacks = [learning_rate_monitor_callback, best_model_checkpoint_callback])

# To test the model's performance on test set.
test_loss, test_metrics = model.evaluate(test_inputs, test_outputs, batch_size = None) # Uncomment these lines to infer model's performance.
print(f'Test Loss: {test_loss}, Test MAPE: {test_metrics}')

# To store the true_emission and predicted emissions for the .csv file.
true_emissions = []
predicted_emissions = []
for i in range(len(test_inputs)):
    inputs = np.expand_dims(test_inputs[i], axis = 0)
    outputs = model.predict(inputs)  # Predicting the output based on the model's learning.
    print("True value : ", test_outputs[i])
    print("Predicted value : ", outputs)
    true_emissions.append(test_outputs[i])  # Append the true emissions into true_emissions list
    predicted_emissions.append(outputs)  # Append the predicted emissions into predicted_emissions list

# Converting into numpy arrays
true_emissions = np.array(true_emissions)  # Shape : 7211, 3
predicted_emissions = np.array(predicted_emissions)  # Shape : 7211, 1, 3
true_emissions = true_emissions.reshape(predicted_emissions.shape)  # After reshaping : 7211, 1, 3

# Storing the true emissions and predicted emissions in a csv file.
df = pd.DataFrame({'True Emissions': true_emissions.flatten(), 'Predicted Emissions': predicted_emissions.flatten()}, index = np.arange(1, len(true_emissions) + 1))
df.to_csv('emiss_results.csv')
print(df)

Error Calculations

In [ ]:
# importing necessary package
import pandas as pd

""" After doing it twice, we now look at error, absolute error, relative errors. """ 
results_df = pd.read_csv('emiss_data.csv') # Reading emiss_data

# Creating new column error which is true-pred.
results_df['error'] = (results_df['True Emissions'] - results_df['Predicted Emissions']).round(3)

# Creating new column absolute_error which is the absolute value of error rounded off to 3 decimal places.
results_df['absolute_error'] = (abs(results_df['error'])).round(3)

# Creating new column relative_error which is the relative value of error rounded off to 3 decimal places with epsilon = 1e-15.
results_df['relative_error'] = ((results_df['error'] / (results_df['True Emissions'] + 1e-15)) * 100).round(3)

# Updates the csv file
results_df.to_csv('emiss_data.csv', index = False)

In [ ]:
# importing necessary package
import pandas as pd

""" After doing it twice, we now look at error, absolute error, relative errors. """ 
results_df = pd.read_csv('emiss_results.csv') # Reading emiss_results

# Creating new column error which is true-pred.
results_df['error'] = (results_df['True Emissions'] - results_df['Predicted Emissions']).round(3)

# Creating new column absolute_error which is the absolute value of error rounded off to 3 decimal places.
results_df['absolute_error'] = (abs(results_df['error'])).round(3)

# Creating new column relative_error which is the relative value of error rounded off to 3 decimal places with epsilon = 1e-15.
results_df['relative_error'] = ((results_df['error'] / (results_df['True Emissions'] + 1e-15)) * 100).round(3)

# Updates the csv file
results_df.to_csv('emiss_results.csv', index = False)

In [ ]:
# importing necessary package
import pandas as pd

df = pd.read_csv('emiss_data.csv') # Reading emiss_data

# We want to know the Q1, Q2, Q3 values or (25%, 50%, 75%) values from our absolute_error distribution.
percentiles = df['absolute_error'].describe(percentiles = [0.25, 0.5, 0.75])

percentiles['25%'] = abs(percentiles['25%'])
percentiles['50%'] = abs(percentiles['50%'])
percentiles['75%'] = abs(percentiles['75%'])

# Only the percentiles are of impportant to us.
percentiles = percentiles.loc[['25%', '50%', '75%']].apply(abs) # 25% : 1.062, 50% : 2.40, 75% : 4.99 (All are in Mt/yr)

# Stated in the paper, the median absolute error is 3 Mt/yr. We got 2.4 Mt/ yr which means we are at a -0.6 Mt/yr deviation which is better than the paper.

# The true percentile values as mentioned in the paper.
true_percentiles = [1.3, 2.7, 4.5] # (All are in Mt/yr)

# Absolute error between our percentile values and the true percentile values rounded off to 3 decimal places.
error = [round(abs(true - pred), 3) for true, pred in zip(true_percentiles, percentiles)] # -0.238, -0.296, 0.49.

# This means that our percentile values are on an average -0.35 Mt/yr range of the true percentile values.
# This is better than the original results as our model is giving us less error.

print(percentiles)
print("Error:", error)

In [ ]:
# importing necessary package
import pandas as pd

df = pd.read_csv('emiss_results.csv') # Reading emiss_results

# We want to know the Q1, Q2, Q3 values or (25%, 50%, 75%) values from our absolute_error distribution.
percentiles = df['absolute_error'].describe(percentiles = [0.25, 0.5, 0.75])

percentiles['25%'] = abs(percentiles['25%'])
percentiles['50%'] = abs(percentiles['50%'])
percentiles['75%'] = abs(percentiles['75%'])

# Only the percentiles are of impportant to us.
percentiles = percentiles.loc[['25%', '50%', '75%']].apply(abs) # 25% : 1.04, 50% : 2.28, 75% : 4.29 (All are in Mt/yr)

# Stated in the paper, the median absolute error is 3 Mt/yr. We got 2.28 Mt/ yr which means we are at a -0.72 Mt/yr deviation which is better than the paper.

# The true percentile values as mentioned in the paper.
true_percentiles = [1.3, 2.7, 4.5] # (All are in Mt/yr)

# Absolute error between our percentile values and the true percentile values rounded off to 3 decimal places.
error = [round(abs(true - pred), 3) for true, pred in zip(true_percentiles, percentiles)] # -0.257, -0.416, -0.204 (All these are negative i.e. our model produced
                                                                                          # better results than the original model.

# This means that our percentile values are on an average -0.35 Mt/yr range of the true percentile values.
# This is better than the original results as our model is giving us less error.

print(percentiles)
print("Error:", error)

In [ ]:
# importing necessary package
import pandas as pd

df = pd.read_csv('emiss_data.csv') # Reading emissions_results
df['abs_rel_error'] = abs(df['relative_error'])

# We want to know the Q1, Q2, Q3 values or (25%, 50%, 75%) values from our relative_error distribution.
percentiles = df['abs_rel_error'].describe()
#percentiles = [0.25, 0.5, 0.75]
# Only the percentiles are of impportant to us.
#percentiles = percentiles.loc[['25%', '50%', '75%']].apply(abs) # 25% : 10.16, 50% : 21, 75% : 38.7 (Unitless)

# Stated in the paper, the median absolute relative paper is around 20%. We have got 21.75% which can be approximated to 22%.

# The true percentile values as mentioned in the paper.
true_percentiles = [8.6, 18.1, 30.3] # Note : These values are percentage values as the relative errors are calculated in terms of percentage.

# Absolute error between our percentile values and the true percentile values rounded off to 3 decimal places.
error = [round(abs(true - pred), 3) for true, pred in zip(true_percentiles, percentiles)] # 1.56, 2.9, 8.4 

#df.to_csv('emiss_data.csv', index = False)
print(percentiles)
print("Error:", error)

In [ ]:
# importing necessary package
import pandas as pd

df = pd.read_csv('emiss_results.csv') # Reading emiss_results
df['abs_rel_error'] = abs(df['relative_error'])

# We want to know the Q1, Q2, Q3 values or (25%, 50%, 75%) values from our relative_error distribution.
percentiles = df['abs_rel_error'].describe(percentiles = [0.25, 0.5, 0.75])

# Only the percentiles are of impportant to us.
percentiles = percentiles.loc[['25%', '50%', '75%']].apply(abs) # 25% : 10.6, 50% : 21, 75% : 33.3 (Unitless)

# Stated in the paper, the median absolute relative paper is around 20%. We have got 21%.
# This means our model performed better.

# The true percentile values as mentioned in the paper.
true_percentiles = [8.6, 18.1, 30.3] # Note : These values are percentage values as the relative errors are calculated in terms of percentage.

# Absolute error between our percentile values and the true percentile values rounded off to 3 decimal places.
error = [round(abs(true - pred), 3) for true, pred in zip(true_percentiles, percentiles)] # 2, 2.9, 3 

df.to_csv('emiss_results.csv', index = False)
print(percentiles)
print("Error:", error)

In [ ]:
# importing necessary packages
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

df1 = pd.read_csv('emiss_results.csv') # Reading emiss_results
df2 = pd.read_csv('emiss_data.csv') # Reading emiss_data

plt.figure(figsize = (10, 6))
sns.kdeplot(df1['absolute_error'], fill = True, color = 'b', label = 'Using U-Net regression') # Using seaborn's kdeplot.
sns.kdeplot(df2['absolute_error'], fill = True, color = 'r', label = 'Using CNN regression')
plt.title('Kernel Density Estimate of Absolute Error for Lippendorf')
plt.xlabel('Absolute Error')
plt.ylabel('Density')
plt.legend()
plt.xlim(0, 25)
#plt.savefig('absolute_error_new.png', dpi = 300, bbox_inches = 'tight') # Saving the file
plt.show()

plt.figure(figsize = (10, 6))
sns.kdeplot(df1['relative_error'], fill = True, color = 'b', label = 'Using U-Net regression') # Using seaborn's kdeplot.
sns.kdeplot(df2['relative_error'], fill = True, color = 'r', label = 'Using CNN regression')
plt.title('Kernel Density Estimate of Relative Error (%) for Lippendorf')
plt.xlabel('Relative Error (%)')
plt.ylabel('Density')
plt.legend()
plt.xlim(-180, 180)
#plt.savefig('relative_error_new.png', dpi = 300, bbox_inches = 'tight') # Saving the file
plt.show()

In [ ]:
""" This code is to know how many of our absolute errors lie in (0-2) Mt/yr range, in (2-5) Mt/yr range, in (5-10) Mt/yr range
and in 10 Mt/yr above. Along with that, let us also get some statistics. """

# importing necessary package
import pandas as pd

# Read csv file
df = pd.read_csv('emiss_results.csv')

# useful column
abs = df['absolute_error']

# get the required info
in0_2 = (abs >= 0) & (abs <= 2)
in2_5 = (abs > 2) & (abs <= 5)
in5_10 = (abs > 5) & (abs <= 10)
ab10 = abs > 10
mean = round(abs.mean(), 3)
median = round(abs.median(), 3)
std = round(abs.std(), 3)

print("in between 0 and 2 : ", in0_2.sum()) # 3219
print("in between 2 and 5 : ", in2_5.sum()) # 2509
print("in between 5 and 10 : ", in5_10.sum()) # 1099
print("above 10 : ", ab10.sum()) # 384
print("mean : ", mean) # 2.9
print("median : ", median) # 2.28
print("std : ", std) # 3.3

# According to the paper, " Majority of errors are concentrated below 10 Mt/yr." which we can see is true. In our case, we can say that majority of errors are
# below 5 Mt/yr which is a significant increase in performance from 10 Mt/yr.

In [ ]:
""" This code is to know how many of our absolute errors lie in (0-2) Mt/yr range, in (2-5) Mt/yr range, in (5-10) Mt/yr range
and in 10 Mt/yr above. Along with that, let us also get some statistics. """

# importing necessary package
import pandas as pd

# Read csv file
df = pd.read_csv('emiss_data.csv')

# useful column
abs = df['absolute_error']

# get the required info
in0_2 = (abs >= 0) & (abs <= 2)
in2_5 = (abs > 2) & (abs <= 5)
in5_10 = (abs > 5) & (abs <= 10)
ab10 = abs > 10
mean = round(abs.mean(), 3)
median = round(abs.median(), 3)
std = round(abs.std(), 3)

print("in between 0 and 2 : ", in0_2.sum()) # 3143
print("in between 2 and 5 : ", in2_5.sum()) # 2267
print("in between 5 and 10 : ", in5_10.sum()) # 1305
print("above 10 : ", ab10.sum()) # 496
print("mean : ", mean) # 3.6
print("median : ", median) # 2.4
print("std : ", std) # 3.75

# According to the paper, " Majority of errors are concentrated below 10 Mt/yr." which we can see is true. In our case, we can say that majority of errors are
# below 5 Mt/yr which is a significant increase in performance from 10 Mt/yr.

In [ ]:
""" This code is to know how many of our relative errors lie less than -150%, in (-150 to -100)% range, in (-100 to -50)% range,
in (-50 to 0) %, in (0 to 50) % range, in (50 to 100) % and above 100%. Along with that, let us also get some statistics. """

# importing necessary package
import pandas as pd

# Read csv file
df = pd.read_csv('emiss_data.csv')

# useful column
rel = df['relative_error']

# get the required info
lessneg150 = (rel <= -150)
neg150_100 = (rel > -150) & (rel <= -100)
neg100_50 = (rel > -100) & (rel <= -50)
neg50_0 = (rel > -50) & (rel <= 0)
in0_50 = (rel > 0) & (rel <= 50)
in50_100 = (rel > 50) & (rel <= 100)
ab100 = rel > 100
mean = round(rel.mean(), 3)
median = round(rel.median(), 3)
std = round(rel.std(), 3)

print("less than -150% : ", lessneg150.sum()) # 104
print("in between -150% and -100% : ", neg150_100.sum()) # 202
print("in between -100% and -50% : ", neg100_50.sum()) # 789
print("in between -50% and 0% : ", neg50_0.sum()) # 2782
print("in between 0% and 50% : ", in0_50.sum()) # 3249
print("in between 50% and 100% : ", in50_100.sum()) # 145
print("above 100% : ", ab100.sum()) # 0
print("mean : ", mean) # -11.081
print("median : ", median) # -2.5
print("std : ", std) # 43.5

In [ ]:
""" This code is to know how many of our relative errors lie less than -150%, in (-150 to -100)% range, in (-100 to -50)% range,
in (-50 to 0) %, in (0 to 50) % range, in (50 to 100) % and above 100%. Along with that, let us also get some statistics. """

# importing necessary package
import pandas as pd

# Read csv file
df = pd.read_csv('emiss_results.csv')

# useful column
rel = df['relative_error']

# get the required info
lessneg150 = (rel <= -150)
neg150_100 = (rel > -150) & (rel <= -100)
neg100_50 = (rel > -100) & (rel <= -50)
neg50_0 = (rel > -50) & (rel <= 0)
in0_50 = (rel > 0) & (rel <= 50)
in50_100 = (rel > 50) & (rel <= 100)
ab100 = rel > 100
mean = round(rel.mean(), 3)
median = round(rel.median(), 3)
std = round(rel.std(), 3)

print("less than -150% : ", lessneg150.sum()) # 56
print("in between -150% and -100% : ", neg150_100.sum()) # 96
print("in between -100% and -50% : ", neg100_50.sum()) # 319
print("in between -50% and 0% : ", neg50_0.sum()) # 1938
print("in between 0% and 50% : ", in0_50.sum()) # 4633
print("in between 50% and 100% : ", in50_100.sum()) # 169
print("above 100% : ", ab100.sum()) # 0
print("mean : ", mean) # 3.818
print("median : ", median) # 11.6
print("std : ", std) # 36.8

In [ ]:
""" This code is to know how many of our absolute relative errors lie in (0 to 20) % range, in (20 to 50)% range, in (50 to 100) %, in (100 to 150)% 
and above 150%. Along with that, let us also get some statistics. """

# importing necessary package
import pandas as pd

# Read csv file
df = pd.read_csv('emiss_data.csv')

# useful column
rel = df['abs_rel_error']

# get the required info
above150 = (rel > 150)
in150_100 = (rel > 100) & (rel <= 150)
in100_50 = (rel > 50) & (rel <= 100)
in50_20 = (rel > 20) & (rel <= 50)
in20_0 = (rel > 0) & (rel <= 20)

mean = round(rel.mean(), 3)
median = round(rel.median(), 3)
std = round(rel.std(), 3)

print("above 150% : ", above150.sum()) # 104
print("in between 100% and 150% : ", in150_100.sum()) # 202
print("in between 50% and 100% : ", in100_50.sum()) # 874
print("in between 20% and 50% : ", in50_20.sum()) # 2676
print("in between 0% and 20% : ", in20_0.sum()) # 3355
print("mean : ", mean) # 30.727
print("median : ", median) # 22
print("std : ", std) # 32.7

# According to the paper, " Majority of errors are concentrated below 10 Mt/yr or 50% with very few exceeding 100%." which we can again see is true
# In our model also, Majority of the errors are below 50%.

In [ ]:
""" This code is to know how many of our absolute relative errors lie in (0 to 20) % range, in (20 to 50)% range, in (50 to 100) %, in (100 to 150)% 
and above 150%. Along with that, let us also get some statistics. """

# importing necessary package
import pandas as pd

# Read csv file
df = pd.read_csv('emiss_results.csv')

# useful column
rel = df['abs_rel_error']

# get the required info
above150 = (rel > 150)
in150_100 = (rel > 100) & (rel <= 150)
in100_50 = (rel > 50) & (rel <= 100)
in50_20 = (rel > 20) & (rel <= 50)
in20_0 = (rel > 0) & (rel <= 20)

mean = round(rel.mean(), 3)
median = round(rel.median(), 3)
std = round(rel.std(), 3)

print("above 150% : ", above150.sum()) # 56
print("in between 100% and 150% : ", in150_100.sum()) # 96
print("in between 50% and 100% : ", in100_50.sum()) # 487
print("in between 20% and 50% : ", in50_20.sum()) # 3127
print("in between 0% and 20% : ", in20_0.sum()) # 3444
print("mean : ", mean) # 26
print("median : ", median) # 21.02
print("std : ", std) # 26.2

# According to the paper, " Majority of errors are concentrated below 10 Mt/yr or 50% with very few exceeding 100%." which we can again see is true
# In our model also, Majority of the errors are below 50%.

In [ ]:
""" This code gets us a few statistics about true emissions and predicted emissions. """

# necessary package
import pandas as pd

# read
df = pd.read_csv('emiss_data.csv')

true = df['True Emissions']
min_true = round(true.min(), 3)
max_true = round(true.max(), 3)
mean_true = round(true.mean(), 3)
std_true = round(true.std(), 3)
median_true = round(true.median(), 3)
range_true = round(max_true-min_true, 3)

pred = df['Predicted Emissions']
min_pred = round(pred.min(), 3)
max_pred = round(pred.max(), 3)
mean_pred = round(pred.mean(), 3)
std_pred = round(pred.std(), 3)
median_pred = round(pred.median(), 3)
range_pred = round(max_pred-min_pred, 3)

print("True statistics : ")
print()
print("min : ", min_true) # 2.861
print("max : ", max_true) # 52.659
print("mean : ", mean_true) # 13.534
print("std : ", std_true) # 8.772
print("median : ", median_true) # 10.27
print("range : ", range_true) # 49.798
print()
print("Predicted statistics : ")
print()
print("min : ", min_pred) # 4.347
print("max : ", max_pred) # 46.892
print("mean : ", mean_pred) # 12.479
print("std : ", std_pred) # 7.692
print("median : ", median_pred) # 11.123
print("range : ", range_pred) # 42.545

# The mean true emission = 13.5 Mt/yr and mean pred emission = 12.48 Mt/yr which tantamounts to the fact that our model has come closer to the real true emission values.

In [ ]:
""" This code gets us a few statistics about true emissions and predicted emissions. """

# necessary package
import pandas as pd

# read
df = pd.read_csv('emiss_results.csv')

true = df['True Emissions']
min_true = round(true.min(), 3)
max_true = round(true.max(), 3)
mean_true = round(true.mean(), 3)
std_true = round(true.std(), 3)
median_true = round(true.median(), 3)
range_true = round(max_true-min_true, 3)

pred = df['Predicted Emissions']
min_pred = round(pred.min(), 3)
max_pred = round(pred.max(), 3)
mean_pred = round(pred.mean(), 3)
std_pred = round(pred.std(), 3)
median_pred = round(pred.median(), 3)
range_pred = round(max_pred-min_pred, 3)

print("True statistics : ")
print()
print("min : ", min_true) # 2.875
print("max : ", max_true) # 52.659
print("mean : ", mean_true) # 13.672
print("std : ", std_true) # 8.982
print("median : ", median_true) # 10.294
print("range : ", range_true) # 49.784
print()
print("Predicted statistics : ")
print()
print("min : ", min_pred) # 2.748
print("max : ", max_pred) # 52.902
print("mean : ", mean_pred) # 13.459
print("std : ", std_pred) # 8.02
print("median : ", median_pred) # 9.264
print("range : ", range_pred) # 50.154

# The mean true emission = 13.67 Mt/yr and mean pred emission = 13.5 Mt/yr which tantamounts to the fact that our model has come closer to the real true emission values.

In [ ]:
# importing necessary packages
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

df1 = pd.read_csv('emiss_results_lip.csv') # Reading emiss_results
df2 = pd.read_csv('emiss_data_lip.csv') # Reading emiss_data
df3 = pd.read_csv('emissions_data_lip.csv')
df4 = pd.read_csv('emissions_results_lip.csv')

plt.figure(figsize = (10, 6))
sns.kdeplot(df1['absolute_error'], fill = True, color = 'b', label = 'Using U-Net regression on curated dataset') # Using seaborn's kdeplot.
sns.kdeplot(df2['absolute_error'], fill = True, color = 'r', label = 'Using CNN regression on curated dataset')
sns.kdeplot(df3['absolute_error'], fill = True, color = 'g', label = 'Using CNN regression on original dataset')
sns.kdeplot(df4['absolute_error'], fill = True, color = 'yellow', label = 'Using U-Net regression on original dataset')
plt.title('Kernel Density Estimate of Absolute Error for Lippendorf')
plt.xlabel('Absolute Error')
plt.ylabel('Density')
plt.legend()
plt.xlim(0, 25)
plt.savefig('absolute_error_comb.png', dpi = 300, bbox_inches = 'tight') # Saving the file
plt.show()

plt.figure(figsize = (10, 6))
sns.kdeplot(df1['relative_error'], fill = True, color = 'b', label = 'Using U-Net regression on curated dataset') # Using seaborn's kdeplot.
sns.kdeplot(df2['relative_error'], fill = True, color = 'r', label = 'Using CNN regression on curated dataset')
sns.kdeplot(df3['relative_error'], fill = True, color = 'g', label = 'Using CNN regression on original dataset')
sns.kdeplot(df4['relative_error'], fill = True, color = 'yellow', label = 'Using U-Net regression on original dataset')
plt.title('Kernel Density Estimate of Relative Error (%) for Lippendorf')
plt.xlabel('Relative Error (%)')
plt.ylabel('Density')
plt.legend()
plt.xlim(-180, 180)
plt.savefig('relative_error_comb.png', dpi = 300, bbox_inches = 'tight') # Saving the file
plt.show()